In [3]:
import requests
import pandas as pd
import numpy as np
!pip install riotwatcher
from riotwatcher._apis.league_of_legends import MatchApiV5
from riotwatcher import LolWatcher, ApiError
import numpy as np

In [4]:
def get_match_json(matchid):
    api_key='RGAPI-5e560ee8-ffc9-440c-b7b3-950c07e889dc'
    region='AMERICAS'
    
    url_pull_match = "https://{}.api.riotgames.com/lol/match/v5/matches/{}/timeline?api_key={}".format(region, matchid, api_key)
    match_data_all = requests.get(url_pull_match).json()

    #try:
     #   length_match = match_data_all['info']['frames'][15]
    return match_data_all
   # except IndexError:
    #    return ['Match is too short. Skipping.']


In [5]:
def subtract_list(list1,list2):
    array1 = np.array(list1)
    array2 = np.array(list2)
    subtracted_array = np.subtract(array1, array2)
    subtracted = list(subtracted_array)
    return subtracted


In [6]:
def cal_gold_dif(golddiff,data,time):
    team1gold=0
    team2gold=0
    for i in [1,2,3,4,5]:
        team1gold+=get_player_stats(data, i,time)
    for j in [6,7,8,9,10]:
        team2gold+=get_player_stats(data, i,time)
    golddiff.append(team1gold-team2gold)
    return golddiff
def get_player_stats(match_data, player,time): #  player has to be an int (1-10)
    # Get player information 

    player_query = match_data['info']['frames'][time]['participantFrames'][str(player)]
    #player_team = player_query['teamId'] - It is not possibly with the endpoint to get the teamId
    player_total_gold = player_query['totalGold']
    
    
    return player_total_gold

def new_pad_list(mylist,time):
    #pad un-updated list with the previous value
    if len(mylist) <(time+1):
        mylist.append(mylist[-1])
    return mylist

def append_1min_stat(team1drag,team2drag,team1baron,team2baron,team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib,team1kill,team2kill,df_1min,time):
   #count kc for kill count, tc for turrent count , ic for inhibitor count
    team1kc=0
    team2kc=0
    team1tc=0
    team2tc=0
    team1ic=0
    team2ic=0
    winner=0
    for i in range(df_1min.shape[0]):
        #champion kill count
        if df_1min['type'].iloc[i]=='CHAMPION_KILL':
            if df_1min['killerId'].iloc[i] in [1,2,3,4,5]:
                team1kc+=1
            else:
                team2kc+=1
        #dragon kill,baron kill, herald kill
        if df_1min['type'].iloc[i]=='ELITE_MONSTER_KILL':
            if df_1min['monsterType'].iloc[i]=='DRAGON':
                if df_1min['killerTeamId'].iloc[i]== 100.0:
                    team1drag.append(team1drag[-1]+1)  
                
                if df_1min['killerTeamId'].iloc[i]==200.0:
                    team2drag.append(team2drag[-1]+1)
            if df_1min['monsterType'].iloc[i]=='BARON_NASHOR':
                if df_1min['killerTeamId'].iloc[i]== 100.0:
                    team1baron.append(team1baron[-1]+1)  
                
                if df_1min['killerTeamId'].iloc[i]==200.0:
                    team2baron.append(team2baron[-1]+1)
            if df_1min['monsterType'].iloc[i]=='RIFTHERALD':
                if df_1min['killerTeamId'].iloc[i]== 100.0:
                    team1herald.append(team1herald[-1]+1)  
                
                if df_1min['killerTeamId'].iloc[i]==200.0:
                    team2herald.append(team2herald[-1]+1) 
        #building kill:turrent and inhibitors
        if df_1min['type'].iloc[i]=='BUILDING_KILL':
            if df_1min['buildingType'].iloc[i]=='TOWER_BUILDING':
                if df_1min['killerId'].iloc[i] in [1,2,3,4,5]:
                    team1tc+=1
                else:
                    team2tc+=1
            if df_1min['buildingType'].iloc[i]=='INHIBITOR_BUILDING':
                if df_1min['killerId'].iloc[i] in [1,2,3,4,5]:
                    team1ic+=1
                else:
                    team2ic+=1
        if df_1min['type'].iloc[i]=='GAME_END':
            if df_1min['winningTeam'].iloc[i]==100.0:
                winner=0
            else:
                winner=1
    #append new champion kill, turrent kill, inhibitor kill 
    team1kill.append(team1kill[-1]+team1kc)
    team2kill.append(team2kill[-1]+team2kc)
    team1turrent.append(team1turrent[-1]+team1tc)
    team2turrent.append(team2turrent[-1]+team2tc)
    team1inhib.append(team1inhib[-1]+team1ic)
    team2inhib.append(team2inhib[-1]+team2ic)
    team1drag=new_pad_list(team1drag,time)
    team2drag=new_pad_list(team2drag,time)
    team1baron=new_pad_list(team1baron,time)
    team2baron=new_pad_list(team2baron,time)
    team1herald=new_pad_list(team1herald,time)
    team2herald=new_pad_list(team2herald,time)
    
    return winner,team1drag,team2drag,team1baron,team2baron,team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib,team1kill,team2kill

In [7]:
def get_1match_data(matchid):
    outputdf=pd.DataFrame(columns=['golddiff','dragondiff','barondiff','heralddiff','towerdiff','inhibitordiff','killdiff','winner'])
    count=0
    for _matchid in matchid:
        json=get_match_json("NA1_"+str(_matchid))
       
        json2=json['info']['frames']
        gamelength=len(json2)+1
        team1drag=[0]
        team2drag=[0]
        team1baron=[0]
        team2baron=[0]
        team1herald=[0]
        team2herald=[0]
        team1turrent=[0]
        team2turrent=[0]
        team1inhib=[0]
        team2inhib=[0]
        team1kill=[0]
        team2kill=[0]
        golddiff=[0]
        for i in range(gamelength):
            event_query = json['info']['frames'][i]['events']
    
            df=pd.DataFrame.from_dict(event_query)
            winner,team1drag,team2drag,team1baron,team2baron,team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib,team1kill,team2kill=append_1min_stat(
            team1drag,team2drag,team1baron,team2baron,team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib,team1kill,team2kill,df,(i+1))
            golddiff=cal_gold_dif(golddiff,json,i)
        df2=pd.DataFrame([[golddiff,subtract_list(team1drag,team2drag),subtract_list(team1baron,team2baron),subtract_list(team1herald,team2herald),
                         subtract_list(team1turrent,team2turrent),subtract_list(team1inhib,team2inhib),subtract_list(team1kill,team2kill),winner ]],columns=['golddiff','dragondiff','barondiff','heralddiff','towerdiff','inhibitordiff','killdiff','winner'])
        outputdf=pd.concat([outputdf,df2],ignore_index=True)
        count+=1
        print("finish:",count)

    return outputdf
        

In [8]:
def get_1match_datav2(matchid):
    outputdf=pd.DataFrame(columns=['golddiff','dragondiff','barondiff','heralddiff','towerdiff','inhibitordiff','killdiff'])
    count=0
    for _matchid in matchid:
     
        json=get_match_json("NA1_"+str(_matchid))
        if (json =={'status': {'message': 'Data not found - match file not found',
  'status_code': 404}} or json=={'status': {'message': 'Forbidden', 'status_code': 403}}):
            continue
        else:
        
            json2=json['info']['frames']
       
            gamelength=len(json2)
            team1drag=[0]
            team2drag=[0]
            team1baron=[0]
            team2baron=[0]
            team1herald=[0]
            team2herald=[0]
            team1turrent=[0]
            team2turrent=[0]
            team1inhib=[0]
            team2inhib=[0]
            team1kill=[0]
            team2kill=[0]
            golddiff=[0]
       
            for i in range(gamelength):
                event_query = json['info']['frames'][i]['events']
    
                df=pd.DataFrame.from_dict(event_query)
                winner,team1drag,team2drag,team1baron,team2baron,team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib,team1kill,team2kill=append_1min_stat(
                team1drag,team2drag,team1baron,team2baron,team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib,team1kill,team2kill,df,(i+1))
                golddiff=cal_gold_dif(golddiff,json,i)
            df2=pd.DataFrame([[golddiff,subtract_list(team1drag,team2drag),subtract_list(team1baron,team2baron),subtract_list(team1herald,team2herald),
                         subtract_list(team1turrent,team2turrent),subtract_list(team1inhib,team2inhib),subtract_list(team1kill,team2kill),winner ]],columns=['golddiff','dragondiff','barondiff','heralddiff','towerdiff','inhibitordiff','killdiff','winner'])
            outputdf=pd.concat([outputdf,df2],ignore_index=True)
            count+=1
            print("finish:",count)
        
    return outputdf
        

In [18]:
def get_1match_visdata(matchid):
    outputdf=pd.DataFrame(columns=['golddiff','dragondiff','barondiff','heralddiff','towerdiff','inhibitordiff','bluedragon','reddragon','bluebaron','redbaron','blueherald','redherald','bluetower','redtower','blueinhhib','redinhib'])
    #outputdf2=pd.DataFrame(colums=['bluedragon','reddragon','bluebaron','redbaron','blueherald','redherald','bluetower','redtower','blueinhhib','redinhib','winner'])
    count=0
    for _matchid in matchid:
     
        json=get_match_json("NA1_"+str(_matchid))
        if (json =={'status': {'message': 'Data not found - match file not found',
  'status_code': 404}} or json=={'status': {'message': 'Forbidden', 'status_code': 403}}):
            continue
        else:
        
            json2=json['info']['frames']
       
            gamelength=len(json2)
            team1drag=[0]
            team2drag=[0]
            team1baron=[0]
            team2baron=[0]
            team1herald=[0]
            team2herald=[0]
            team1turrent=[0]
            team2turrent=[0]
            team1inhib=[0]
            team2inhib=[0]
            team1kill=[0]
            team2kill=[0]
            golddiff=[0]
       
            for i in range(gamelength):
                event_query = json['info']['frames'][i]['events']
    
                df=pd.DataFrame.from_dict(event_query)
                winner,team1drag,team2drag,team1baron,team2baron,team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib,team1kill,team2kill=append_1min_stat(
                team1drag,team2drag,team1baron,team2baron,team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib,team1kill,team2kill,df,(i+1))
                golddiff=cal_gold_dif(golddiff,json,i)
            df2=pd.DataFrame([[golddiff,subtract_list(team1drag,team2drag),subtract_list(team1baron,team2baron),subtract_list(team1herald,team2herald),
                         subtract_list(team1turrent,team2turrent),subtract_list(team1inhib,team2inhib),subtract_list(team1kill,team2kill),winner,team1drag,team2drag,team1baron,team2baron,
            team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib                  
                              ]],columns=['golddiff','dragondiff','barondiff','heralddiff','towerdiff','inhibitordiff','killdiff','winner','bluedragon','reddragon','bluebaron','redbaron','blueherald','redherald','bluetower','redtower','blueinhhib','redinhib'])
            outputdf=pd.concat([outputdf,df2],ignore_index=True)
            count+=1
            print("finish:",count)
        
    return outputdf
        

In [ ]:
df2=pd.DataFrame([[team1drag,team2drag,team1baron,team2baron,team1herald,team2herald,team1turrent,team2turrent,team1inhib,team2inhib,team1kill,team2kill,golddiff]],columns=['bdragons','rdragons','bbarons','rbarons','bheralds','rheralds','bturrents','rturrents','binhib','rinhib','bkills','rkills','golddiff'])

In [10]:
matchidlist=[4035079604, 4035079613, 4035079665, 4035079666, 4035079668, 4035079686, 4035079692, 4035079696, 4035079700, 4035079718, 4035079733, 4035089527, 4035089567, 4035089568, 4035089573, 4035089579, 4035089582, 4035089606, 4035089614, 4035089624, 4035089626, 4035089632, 4035089659, 4035089661, 4035089662, 4035089666, 4035089690, 4035089707, 4035089711, 4035089723, 4035089726, 4035098534, 4035098537, 4035098619, 4035098637, 4035098655, 4035098658, 4035098660, 4035098675, 4035098702, 4035098709, 4035098711, 4035098724, 4035098732, 4035098733, 4035098734, 4035098778, 4035098785, 4035098786, 4035098799, 4035098805, 4035098818, 4035098855, 4035098860, 4035098870, 4035098873, 4035098879, 4035098884, 4035098890, 4035098893, 4035098897, 4035098898, 4035098913, 4035098925, 4035098937, 4035098938, 4035098957, 4035098967, 4035098968, 4035098977, 4035098999, 4035099000, 4035099001, 4035099011, 4035099026, 4035099035, 4035099036, 4035099037, 4035099057, 4035099061, 4035099064, 4035099065, 4035099069, 4035099089, 4035099107, 4035099109, 4035099114, 4035099129, 4035099132, 4035099133, 4035099162, 4035099172, 4035099182, 4035099192, 4035099203, 4035099220, 4035099226, 4035099230, 4035099234, 4035099239, 4035099242, 4035099246, 4035099247, 4035099256, 4035099257, 4035099262, 4035099279, 4035099329, 4035099335, 4035099363, 4035099409, 4035099421, 4035099425, 4035099437, 4035099446, 4035099447, 4035099456, 4035099458, 4035099496, 4035099508, 4035099514, 4035099519, 4035099527, 4035099540, 4035099552, 4035099561, 4035099570, 4035099578, 4035099585, 4035099593, 4035099606, 4035099619, 4035099623, 4035099626, 4035099643, 4035099652, 4035099674, 4035099694, 4035099705, 4035099710, 4035099721, 4035099731, 4035099734, 4035108167, 4035108182, 4035108202, 4035108225, 4035108251, 4035108252, 4035108255, 4035108260, 4035108265, 4035108271, 4035108291, 4035108299, 4035108311, 4035108330, 4035108345, 4035108355, 4035108366, 4035108372, 4035108405, 4035108408, 4035108416, 4035108418, 4035108420, 4035108422, 4035108427, 4035108440, 4035108444, 4035108485, 4035108498, 4035108500, 4035108504, 4035108541, 4035108548, 4035108553, 4035108567, 4035108580, 4035108582, 4035108587, 4035108588, 4035108591, 4035108599, 4035108611, 4035108614, 4035108616, 4035108617, 4035108620, 4035108622, 4035108625, 4035108635, 4035108638, 4035108664, 4035108670, 4035108678, 4035108679, 4035108680, 4035108689, 4035108692, 4035108712, 4035108715, 4035108717, 4035108718, 4035108726, 4035108782, 4035108783, 4035108790, 4035108805, 4035108821, 4035108822, 4035108834, 4035108852, 4035108859, 4035108862, 4035108874, 4035108888, 4035108892, 4035108907, 4035108921, 4035108942, 4035108947, 4035108953, 4035108961, 4035108965, 4035108975, 4035108976, 4035108985, 4035108988, 4035109002, 4035109039, 4035109062, 4035109094, 4035109099, 4035109101, 4035109104, 4035109112, 4035109124, 4035109142, 4035109148, 4035109183, 4035109189, 4035109207, 4035109212, 4035109219, 4035109225, 4035109226, 4035109246, 4035109260, 4035109286, 4035109293, 4035109296, 4035109300, 4035109313, 4035109314, 4035109322, 4035109337, 4035109339, 4035109340, 4035109342, 4035109352, 4035109366, 4035109380, 4035109391, 4035109403, 4035109413, 4035109414, 4035109415, 4035109429, 4035109436, 4035109438, 4035109459, 4035109460, 4035109471, 4035109481, 4035109486, 4035109497, 4035109506, 4035109518, 4035109530, 4035109545, 4035109551, 4035109562, 4035109565, 4035109571, 4035109572, 4035109583, 4035109612, 4035109622, 4035109637, 4035109658, 4035109666, 4035109668, 4035109686, 4035109688, 4035109691, 4035109698, 4035109705, 4035109713, 4035109721, 4035110183, 4035110196, 4035110225, 4035110268, 4035110270, 4035110272, 4035110289, 4035110296, 4035110300, 4035110302, 4035110318, 4035110319, 4035110346, 4035110349, 4035110358, 4035110366, 4035110367, 4035110378, 4035110379, 4035110381, 4035110382, 4035110384, 4035110393, 4035110401, 4035110425, 4035110441, 4035110442, 4035110448, 4035110454, 4035110457, 4035110463, 4035110499, 4035110509, 4035110526, 4035110527, 4035110530, 4035110536, 4035110541, 4035110543, 4035110552, 4035110565, 4035110568, 4035110596, 4035110615, 4035110630, 4035110661, 4035110666, 4035110679, 4035110692, 4035110694, 4035110704, 4035110713, 4035110723, 4035110724, 4035110735, 4035110736, 4035110742, 4035110744, 4035110757, 4035110783, 4035110791, 4035110810, 4035110839, 4035110844, 4035110856, 4035110876, 4035110893, 4035110904, 4035110905, 4035110911, 4035110916, 4035110919, 4035110920, 4035110958, 4035110989, 4035110990, 4035110995, 4035111003, 4035111022, 4035111024, 4035111041, 4035111042, 4035111060, 4035111063, 4035111070, 4035111085, 4035111122, 4035111123, 4035111131, 4035111156, 4035111174, 4035111199, 4035111200, 4035111208, 4035111227, 4035111228, 4035111233, 4035111237, 4035111247, 4035111253, 4035111258, 4035111271, 4035111272, 4035111276, 4035111277, 4035111293, 4035111297, 4035111323, 4035111330, 4035111356, 4035111371, 4035111373, 4035111395, 4035111396, 4035111399, 4035111411, 4035111419, 4035111428, 4035111445, 4035111453, 4035111454, 4035111464, 4035111465, 4035111471, 4035111499, 4035111510, 4035111516, 4035111520, 4035111537, 4035111540, 4035111547, 4035111560, 4035111575, 4035111594, 4035111598, 4035111609, 4035111618, 4035111619, 4035111622, 4035111624, 4035111633, 4035111635, 4035111659, 4035111669, 4035111684, 4035111688, 4035111694, 4035111717, 4035111732, 4035111746, 4035111761, 4035111774, 4035111785, 4035111786, 4035111804, 4035111824, 4035111830, 4035111841, 4035111846, 4035111856, 4035111863, 4035111871, 4035111876, 4035111884, 4035111905, 4035111908, 4035111909, 4035111915, 4035111919, 4035111923, 4035111927, 4035111932, 4035111950, 4035111951, 4035111959, 4035111965, 4035111977, 4035111978, 4035111998, 4035112028, 4035112039, 4035112042, 4035112046, 4035112047, 4035112053, 4035112054, 4035112063, 4035112067, 4035112085, 4035112119, 4035112136, 4035112142, 4035112148, 4035112161, 4035112169, 4035112174, 4035112177, 4035112181, 4035112204, 4035112218, 4035112229, 4035112232, 4035112245, 4035112254, 4035112266, 4035112267, 4035112273, 4035112301, 4035112305, 4035112319, 4035112324, 4035112346, 4035112358, 4035112411, 4035112415, 4035112416, 4035112431, 4035112437, 4035112444, 4035112448, 4035112457, 4035112459, 4035112474, 4035112485, 4035112490, 4035112495, 4035112506, 4035112517, 4035112541, 4035112556, 4035112557, 4035112561, 4035112576, 4035112578, 4035112588, 4035112591, 4035112592, 4035112593, 4035112624, 4035112625, 4035112648, 4035112649, 4035112650, 4035112667, 4035112679, 4035112684, 4035112689, 4035112696, 4035112720, 4035112722, 4035112739, 4035112743, 4035112744, 4035112749, 4035112752, 4035112753, 4035112754, 4035112761, 4035112766, 4035112771, 4035112790, 4035112791, 4035112823, 4035112838, 4035112852, 4035112854, 4035112881, 4035112887, 4035112902, 4035112906, 4035112911, 4035112920, 4035112933, 4035112936, 4035112940, 4035112942, 4035112943, 4035112960, 4035112966, 4035112992, 4035112993, 4035113000, 4035113008, 4035113023, 4035113046, 4035113047, 4035113054, 4035113055, 4035113063, 4035113065, 4035113101, 4035113107, 4035113108, 4035113110, 4035113114, 4035113121, 4035113123, 4035113127, 4035113137, 4035113171, 4035113177, 4035113189, 4035113201, 4035113226, 4035113230, 4035113236, 4035113253, 4035113260, 4035113263, 4035113270, 4035113277, 4035113282, 4035113298, 4035113301, 4035113304, 4035113316, 4035113317, 4035113320, 4035113321, 4035113323, 4035113338, 4035113406, 4035113408, 4035113414, 4035113429, 4035113445, 4035113451, 4035113462, 4035113463, 4035113496, 4035113499, 4035113503, 4035113504, 4035113523, 4035113531, 4035113537, 4035113538, 4035113551, 4035113560, 4035113563, 4035113572, 4035113579, 4035113585, 4035113599, 4035113600, 4035113617, 4035113628, 4035113652, 4035113653, 4035113654, 4035113660, 4035113661, 4035113664, 4035113665, 4035113677, 4035113680, 4035113697, 4035113700, 4035113705, 4035113706, 4035113712, 4035113714, 4035113723, 4035113731, 4035113733, 4035113748, 4035113751, 4035113771, 4035113775, 4035113804, 4035113814, 4035113824, 4035113857, 4035113865, 4035113868, 4035113869, 4035113874, 4035113878, 4035113886, 4035113889, 4035113890, 4035113905, 4035113908, 4035113918, 4035113958, 4035113963, 4035113964, 4035113974, 4035113980, 4035113981, 4035113988, 4035114016, 4035114019, 4035114022, 4035114025, 4035114049, 4035114061, 4035114063, 4035114070, 4035114082, 4035114083, 4035114091, 4035114108, 4035114112, 4035114124, 4035114127, 4035114130, 4035114135, 4035114145, 4035114183, 4035114187, 4035114198, 4035114201, 4035114210, 4035114219, 4035114220, 4035114223, 4035114244, 4035114274, 4035114288, 4035114291, 4035114292, 4035114293, 4035114310, 4035114315, 4035114339, 4035114342, 4035114344, 4035114347, 4035114359, 4035114369, 4035114422, 4035114426, 4035114430, 4035114435, 4035114445, 4035114449, 4035114451, 4035114467, 4035114471, 4035114479, 4035114532, 4035114534, 4035114535, 4035114576, 4035114603, 4035114614, 4035114615, 4035114616, 4035114645, 4035114649, 4035114651, 4035114657, 4035114664, 4035114667, 4035114671, 4035114672, 4035114675, 4035114683, 4035114696, 4035114698, 4035114701, 4035114708, 4035114709, 4035114716, 4035114717, 4035114719, 4035114724, 4035114731, 4035114733, 4035114735, 4035114765, 4035114775, 4035114779, 4035114794, 4035114849, 4035114860, 4035114864, 4035114865, 4035114894, 4035114895, 4035114896, 4035114897, 4035114903, 4035114913, 4035114917, 4035114921, 4035114922, 4035114928, 4035114934, 4035114972, 4035114981, 4035114984, 4035114987, 4035114990, 4035114999, 4035115000, 4035115001, 4035115020, 4035115032, 4035115048, 4035115065, 4035115071, 4035115089, 4035115100, 4035115102, 4035115166, 4035115168, 4035115178, 4035115184, 4035115185, 4035115194, 4035115197, 4035115198, 4035115204, 4035115206, 4035115210, 4035115212, 4035115235, 4035115245, 4035115267, 4035115279, 4035115283, 4035115290, 4035115291, 4035115319, 4035115327, 4035115329, 4035115330, 4035115339, 4035115343, 4035115345, 4035115346, 4035115349, 4035115365, 4035115384, 4035115385, 4035115386, 4035115387, 4035115391, 4035115396, 4035115402, 4035115435, 4035115457, 4035115461, 4035115462, 4035115463, 4035115471, 4035115477, 4035115506, 4035115516, 4035115518, 4035115553, 4035115562, 4035115564, 4035115607, 4035115618, 4035115620, 4035115637, 4035115642, 4035115644, 4035115646, 4035115662, 4035115676, 4035115678, 4035115685, 4035115686, 4035115696, 4035115701, 4035115713, 4035115721, 4035115727, 4035115730, 4035115734, 4035115750, 4035115757, 4035115759, 4035115776, 4035115792, 4035115797, 4035115798, 4035115802, 4035115816, 4035115820, 4035115822, 4035115824, 4035115836, 4035115843, 4035115852, 4035115890, 4035115893, 4035115909, 4035115917, 4035115954, 4035115963, 4035115985, 4035115998, 4035116005, 4035116040, 4035116094, 4035116099, 4035116100, 4035116101, 4035116141, 4035116143, 4035116146, 4035116156, 4035116165, 4035116172, 4035116175, 4035116176, 4035116195, 4035116204, 4035116240, 4035116257, 4035116311, 4035116315, 4035116318, 4035116336, 4035116345, 4035116346, 4035116355, 4035116356, 4035116357, 4035116364, 4035116367, 4035116383, 4035116385, 4035116393, 4035116401, 4035116418, 4035116419, 4035116439, 4035116454, 4035116460, 4035116462, 4035116466, 4035116467, 4035116469, 4035116474, 4035116512, 4035116521, 4035116532, 4035116545, 4035116567, 4035116571, 4035116583, 4035116607, 4035116611, 4035116618, 4035116621, 4035116639, 4035116641, 4035116658, 4035116664, 4035116671, 4035116676, 4035116683, 4035116699, 4035116714, 4035116720, 4035116722, 4035116735, 4035116736, 4035116737, 4035116755, 4035116762, 4035116770, 4035116791, 4035116808, 4035116816, 4035116821, 4035116834, 4035116840, 4035116845, 4035116855, 4035116862, 4035116866, 4035116869, 4035116871, 4035116874, 4035116884, 4035116886, 4035116891, 4035116928, 4035116940, 4035116952, 4035116953, 4035116955, 4035116979, 4035116987, 4035117003, 4035117004, 4035117007, 4035117010, 4035117018, 4035117029, 4035117030, 4035117039, 4035117043, 4035117057, 4035117059, 4035117064, 4035117068, 4035117080, 4035117087, 4035117100, 4035117110, 4035117147, 4035117158, 4035117169, 4035117175, 4035117212, 4035117214, 4035117227, 4035117230, 4035117237, 4035117251, 4035117263, 4035117279, 4035117285, 4035117294, 4035117304, 4035117331, 4035117337, 4035117350, 4035117378, 4035117389, 4035117399, 4035117425, 4035117429, 4035117430, 4035117445, 4035117458, 4035117464, 4035117467, 4035117471, 4035117488, 4035117511, 4035117518, 4035117524, 4035117553, 4035117573, 4035117574, 4035117581, 4035117588, 4035117596, 4035117599, 4035117600, 4035117601, 4035117603, 4035117619, 4035117621, 4035117629, 4035117631, 4035117650, 4035117662, 4035117686, 4035117706, 4035117707, 4035117708, 4035117710, 4035117718, 4035117726, 4035117727, 4035117729, 4035117733, 4035117738, 4035117743, 4035117745, 4035117752, 4035117758, 4035117791, 4035117803, 4035117810, 4035117813, 4035117824, 4035117830, 4035117875, 4035117876, 4035117878, 4035117885, 4035117888, 4035117922, 4035117960, 4035117964, 4035117969, 4035117986, 4035117989, 4035118010, 4035118016, 4035118020, 4035118025, 4035118027, 4035118035, 4035118040, 4035118042, 4035118051, 4035118055, 4035118090, 4035118091, 4035118096, 4035118097, 4035118101, 4035118115, 4035118116, 4035118119, 4035118125, 4035118128, 4035118143, 4035118149, 4035118155, 4035118160, 4035118172, 4035118178, 4035118187, 4035118192, 4035118201, 4035118214, 4035118218, 4035118228, 4035118229, 4035118235, 4035118251, 4035118256, 4035118258, 4035118269, 4035118270, 4035118286, 4035118289, 4035118294, 4035118303, 4035118311, 4035118333, 4035118360, 4035118363, 4035118372, 4035118401, 4035118426, 4035118438, 4035118447, 4035118449, 4035118452, 4035118463, 4035118467, 4035118483, 4035118500, 4035118502, 4035118507, 4035118530, 4035118570, 4035118572, 4035118579, 4035118582, 4035118583, 4035118590, 4035118624, 4035118628, 4035118652, 4035118661, 4035118662, 4035118663, 4035118671, 4035118683, 4035118686, 4035118689, 4035118692, 4035118707, 4035118717, 4035118721, 4035118744, 4035118745, 4035118751, 4035118754, 4035118756, 4035118757, 4035118770, 4035118771, 4035118773, 4035118797, 4035118814, 4035118832, 4035118850, 4035118851, 4035118865, 4035118872, 4035118873, 4035118881, 4035118890, 4035118913, 4035118919, 4035118926, 4035118930, 4035118938, 4035118943, 4035118948, 4035118960, 4035118962, 4035118970, 4035118974, 4035118975, 4035118995, 4035119001, 4035119054, 4035119062, 4035119077, 4035119078, 4035119080, 4035119081, 4035119093, 4035119094, 4035119098, 4035119109, 4035119111, 4035119117, 4035119166, 4035119167, 4035119194, 4035119226, 4035119246, 4035119270, 4035119273, 4035119281, 4035119284, 4035119285, 4035119294, 4035119295, 4035119301, 4035119302, 4035119335, 4035119344, 4035119351, 4035119373, 4035119391, 4035119392, 4035119396, 4035119406, 4035119443, 4035119461, 4035119465, 4035119470, 4035119489, 4035119492, 4035119496, 4035119499, 4035119508, 4035119512, 4035119513, 4035119520, 4035119534, 4035119540, 4035119553, 4035119556, 4035119569, 4035119584, 4035119591, 4035119602, 4035119619, 4035119625, 4035119629, 4035119637, 4035119640, 4035119650, 4035119658, 4035119663, 4035119668, 4035119670, 4035119675, 4035119678, 4035119680, 4035119698, 4035119703, 4035119718, 4035119730, 4035119953, 4035119975, 4035119984, 4035120001, 4035120015, 4035120048, 4035120082, 4035120084, 4035120101, 4035120111, 4035120119, 4035120177, 4035120187, 4035120192, 4035120219, 4035120247, 4035120257, 4035120259, 4035120265, 4035120280, 4035120286, 4035120289, 4035120292, 4035120297, 4035120305, 4035120307, 4035120332, 4035120337, 4035120346, 4035120352, 4035120363, 4035120385, 4035120395, 4035120420, 4035120424, 4035120458, 4035120465, 4035120470, 4035120483, 4035120500, 4035120512, 4035120516, 4035120530, 4035120546, 4035120553, 4035120558, 4035120559, 4035120585, 4035120594, 4035120613, 4035120623, 4035120634, 4035120636, 4035120637, 4035120638, 4035120655, 4035120657, 4035120660, 4035120671, 4035120676, 4035120677, 4035120678, 4035120683, 4035120693, 4035120694, 4035120698, 4035120718, 4035120725, 4035120726, 4035120730, 4035120735, 4035120742, 4035120743, 4035120758, 4035120770, 4035120771, 4035120772, 4035120782, 4035120790, 4035120820, 4035120831, 4035120843, 4035120867, 4035120873, 4035120880, 4035120907, 4035120918, 4035120921, 4035120929, 4035120930, 4035120933, 4035120949, 4035120956, 4035120957, 4035120959, 4035120961, 4035120984, 4035120992, 4035120997, 4035121011, 4035121027, 4035121044, 4035121058, 4035121061, 4035121066, 4035121067, 4035121085, 4035121093, 4035121094, 4035121113, 4035121119, 4035121121, 4035121122, 4035121123, 4035121135, 4035121143, 4035121148, 4035121177, 4035121182, 4035121183, 4035121187, 4035121188, 4035121189, 4035121227, 4035121231, 4035121234, 4035121239, 4035121240, 4035121243, 4035121246, 4035121248, 4035121251, 4035121252, 4035121256, 4035121267, 4035121284, 4035121304, 4035121317, 4035121342, 4035121354, 4035121362, 4035121378, 4035121379, 4035121413, 4035121417, 4035121418, 4035121440, 4035121443, 4035121450, 4035121478, 4035121493, 4035121513, 4035121515, 4035121518, 4035121522, 4035121538, 4035121539, 4035121550, 4035121553, 4035121581, 4035121588, 4035121603, 4035121605, 4035121606, 4035121607, 4035121608, 4035121610, 4035121629, 4035121631, 4035121636, 4035121646, 4035121647, 4035121649, 4035121652, 4035121664, 4035121671, 4035121677, 4035121678, 4035121679, 4035121683, 4035121689, 4035121702, 4035121704, 4035121720, 4035121721, 4035121723, 4035121744, 4035121749, 4035121756, 4035121757, 4035121778, 4035121814, 4035121829, 4035121836, 4035121855, 4035121874, 4035121887, 4035121892, 4035121901, 4035121914, 4035121925, 4035121926, 4035121938, 4035121944, 4035121945, 4035121959, 4035121962, 4035121988, 4035121989, 4035121990, 4035121994, 4035122037, 4035122038, 4035122042, 4035122046, 4035122048, 4035122052, 4035122074, 4035122075, 4035122081, 4035122084, 4035122099, 4035122126, 4035122134, 4035122147, 4035122154, 4035122156, 4035122167, 4035122179, 4035122180, 4035122205, 4035122219, 4035122229, 4035122245, 4035122247, 4035122266, 4035122286, 4035122287, 4035122320, 4035122327, 4035122371, 4035122378, 4035122402, 4035122405, 4035122407, 4035122409, 4035122419, 4035122431, 4035122443, 4035122450, 4035122456, 4035122462, 4035122464, 4035122465, 4035122469, 4035122472, 4035122519, 4035122535, 4035122549, 4035122555, 4035122578, 4035122592, 4035122606, 4035122609, 4035122611, 4035122615, 4035122620, 4035122640, 4035122644, 4035122653, 4035122668, 4035122681, 4035122734, 4035122752, 4035122758, 4035122762, 4035122763, 4035122770, 4035122772, 4035122775, 4035122777, 4035122790, 4035122799, 4035122826, 4035122831, 4035122842, 4035122852, 4035122894, 4035122897, 4035122898, 4035122903, 4035122915, 4035122923, 4035122926, 4035122937, 4035122943, 4035122949, 4035122950, 4035122976, 4035122988, 4035122991, 4035122993, 4035123019, 4035123062, 4035123069, 4035123088, 4035123108, 4035123113, 4035123134, 4035123152, 4035123156, 4035123171, 4035123174, 4035123175, 4035123208, 4035123216, 4035123217, 4035123232, 4035123240, 4035123241, 4035123250, 4035123254, 4035123257, 4035123261, 4035123269, 4035123280, 4035123285, 4035123289, 4035123299, 4035123301, 4035123307, 4035123330, 4035123338, 4035123349, 4035123356, 4035123381, 4035123387, 4035123415, 4035123422, 4035123431, 4035123432, 4035123449, 4035123479, 4035123481, 4035123494, 4035123498, 4035123505, 4035123512, 4035123525, 4035123546, 4035123559, 4035123563, 4035123571, 4035123572, 4035123596, 4035123601, 4035123605, 4035123608, 4035123614, 4035123623, 4035123669, 4035123670, 4035123674, 4035123680, 4035123684, 4035123693, 4035123694, 4035123708, 4035123712, 4035123721, 4035123732, 4035123741, 4035123749, 4035123769, 4035123784, 4035123785, 4035123788, 4035123797, 4035123808, 4035123812, 4035123814, 4035123815, 4035123824, 4035123827, 4035123850, 4035123854, 4035123874, 4035123881, 4035123893, 4035123896, 4035123897, 4035123906, 4035123916, 4035123919, 4035123923, 4035123926, 4035123936, 4035123941, 4035123943, 4035123944, 4035123946, 4035123962, 4035123969, 4035123985, 4035123986, 4035123994, 4035123999, 4035124001, 4035124002, 4035124004, 4035124012, 4035124016, 4035124024, 4035124026, 4035124030, 4035124033, 4035124036, 4035124057, 4035124074, 4035124080, 4035124093, 4035124117, 4035124130, 4035124131, 4035124139, 4035124154, 4035124166, 4035124183, 4035124191, 4035124201, 4035124208, 4035124222, 4035124226, 4035124250, 4035124252, 4035124259, 4035124260, 4035124263, 4035124269, 4035124271, 4035124284, 4035124288, 4035124290, 4035124292, 4035124304, 4035124314, 4035124329, 4035124335, 4035124341, 4035124343, 4035124358, 4035124363, 4035124366, 4035124395, 4035124411, 4035124422, 4035124430, 4035124437, 4035124445, 4035124449, 4035124455, 4035124494, 4035124495, 4035124515, 4035124528, 4035124562, 4035124563, 4035124602, 4035124604, 4035124606, 4035124620, 4035124624, 4035124630, 4035124633, 4035124639, 4035124646, 4035124665, 4035124674, 4035124686, 4035124691, 4035124692, 4035124694, 4035124750, 4035124764, 4035124773, 4035124774, 4035124777, 4035124782, 4035124785, 4035124788, 4035124795, 4035124798, 4035124804, 4035124807, 4035124808, 4035124827, 4035124828, 4035124829, 4035124840, 4035124862, 4035124865, 4035124870, 4035124935, 4035124944, 4035124947, 4035124949, 4035124960, 4035124964, 4035124989, 4035124997, 4035125000, 4035125011, 4035125025, 4035125028, 4035125029, 4035125037, 4035125058, 4035125069, 4035125070, 4035125072, 4035125079, 4035125090, 4035125105, 4035125112, 4035125114, 4035125121, 4035125137, 4035125142, 4035125143, 4035125153, 4035125155, 4035125185, 4035125189, 4035125218, 4035125232, 4035125255, 4035125261, 4035125263, 4035125270, 4035125279, 4035125280, 4035125284, 4035125286, 4035125290, 4035125293, 4035125300, 4035125315, 4035125323, 4035125326, 4035125338, 4035125340, 4035125347, 4035125353, 4035125354, 4035125355, 4035125360, 4035125372, 4035125378, 4035125382, 4035125402, 4035125403, 4035125407, 4035125445, 4035125448, 4035125449, 4035125465, 4035125472, 4035125483, 4035125489, 4035125493, 4035125505, 4035125538, 4035125552, 4035125557, 4035125565, 4035125566, 4035125574, 4035125587, 4035125591, 4035125613, 4035125622, 4035125624, 4035125634, 4035125642, 4035125648, 4035125651, 4035125662, 4035125681, 4035125696, 4035125702, 4035125704, 4035125706, 4035125713, 4035125717, 4035125725, 4035125734, 4035125746, 4035125747, 4035125749, 4035125753, 4035125771, 4035125773, 4035125791, 4035125792, 4035125807, 4035125812, 4035125813, 4035125824, 4035125833, 4035125841, 4035125846, 4035125851, 4035125856, 4035125863, 4035125871, 4035125872, 4035125897, 4035125910, 4035125917, 4035125952, 4035125965, 4035125969, 4035125982, 4035125991, 4035126001, 4035126013, 4035126016, 4035126020, 4035126048, 4035126054, 4035126064, 4035126094, 4035126095, 4035126106, 4035126122, 4035126128, 4035126141, 4035126155, 4035126159, 4035126171, 4035126181, 4035126182, 4035126216, 4035126217, 4035126227, 4035126235, 4035126243, 4035126250, 4035126257, 4035126277, 4035126279, 4035126280, 4035126285, 4035126287, 4035126289, 4035126298, 4035126306, 4035126314, 4035126319, 4035126321, 4035126327, 4035126332, 4035126344, 4035126351, 4035126364, 4035126367, 4035126371, 4035126381, 4035126382, 4035126391, 4035126414, 4035126453, 4035126465, 4035126505, 4035126533, 4035126545, 4035126553, 4035126555, 4035126560, 4035126573, 4035126579, 4035126599, 4035126615, 4035126627, 4035126631, 4035126638, 4035126642, 4035126646, 4035126650, 4035126660, 4035126662, 4035126676, 4035126702, 4035126723, 4035126732, 4035126750, 4035126767, 4035126769, 4035126778, 4035126797, 4035126798, 4035126817, 4035126825, 4035126877, 4035126880, 4035126886, 4035126890, 4035126896, 4035126898, 4035126909, 4035126910, 4035126917, 4035126923, 4035126926, 4035126931, 4035126938, 4035126952, 4035126956, 4035126969, 4035126973, 4035126974, 4035126977, 4035126987, 4035126988, 4035126993, 4035127007, 4035127012, 4035127015, 4035127017, 4035127024, 4035127026, 4035127034, 4035127035, 4035127041, 4035127079, 4035127093, 4035127097, 4035127104, 4035127120, 4035127130, 4035127131, 4035127141, 4035127171, 4035127185, 4035127197, 4035127203, 4035127204, 4035127213, 4035127214, 4035127219, 4035127222, 4035127238, 4035127239, 4035127244, 4035127249, 4035127282, 4035127287, 4035127306, 4035127326, 4035127329, 4035127346, 4035127373, 4035127383, 4035127384, 4035127395, 4035127400, 4035127403, 4035127409, 4035127436, 4035127450, 4035127486, 4035127487, 4035127505, 4035127509, 4035127524, 4035127546, 4035127560, 4035127624, 4035127628, 4035127638, 4035127641, 4035127642, 4035127644, 4035127648, 4035127670, 4035127678, 4035127698, 4035127712, 4035127713, 4035127721, 4035127741, 4035127744, 4035127761, 4035127769, 4035127779, 4035127780, 4035127787, 4035127789, 4035127791, 4035127797, 4035127804, 4035127810, 4035127828, 4035127839, 4035127858, 4035127860, 4035127863, 4035127865, 4035127884, 4035127889, 4035127890, 4035127895, 4035127896, 4035127908, 4035127915, 4035127916, 4035127926, 4035127933, 4035127942, 4035127950, 4035127987, 4035127994, 4035128009, 4035128019, 4035128044, 4035128071, 4035128074, 4035128087, 4035128090, 4035128106, 4035128113, 4035128134, 4035128139, 4035128148, 4035128153, 4035128154, 4035128173, 4035128175, 4035128195, 4035128196, 4035128229, 4035128236, 4035128237, 4035128246, 4035128262, 4035128267, 4035128268, 4035128277, 4035128278, 4035128283, 4035128305, 4035128306, 4035128312, 4035128313, 4035128326, 4035128336, 4035128340, 4035128351, 4035128359, 4035128362, 4035128366, 4035128370, 4035128379, 4035128391, 4035128392, 4035128396, 4035128397, 4035128428, 4035128452, 4035128458, 4035128468, 4035128471, 4035128477, 4035128484, 4035128499, 4035128500, 4035128502, 4035128510, 4035128514, 4035128529, 4035128546, 4035128549, 4035128558, 4035128567, 4035128585, 4035128603, 4035128615, 4035128619, 4035128633, 4035128642, 4035128646, 4035128656, 4035128658, 4035128662, 4035128667, 4035128671, 4035128672, 4035128705, 4035128714, 4035128737, 4035128740, 4035128755, 4035128773, 4035128774, 4035128781, 4035128782, 4035128789, 4035128799, 4035128810, 4035128816, 4035128838, 4035128843, 4035128850, 4035128876, 4035128877, 4035128888, 4035128902, 4035128905, 4035128917, 4035128919, 4035128954, 4035128965, 4035128971, 4035128981, 4035128990, 4035128997, 4035129001, 4035129012, 4035129031, 4035129034, 4035129038, 4035129039, 4035129046, 4035129064, 4035129084, 4035129110, 4035129121, 4035129123, 4035129130, 4035129134, 4035129170, 4035129189, 4035129194, 4035129197, 4035129203, 4035129211, 4035129218, 4035129220, 4035129229, 4035129235, 4035129267, 4035129269, 4035129296, 4035129311, 4035129312, 4035129316, 4035129336, 4035129371, 4035129387, 4035129390, 4035129421, 4035129422, 4035129429, 4035129448, 4035129455, 4035129459, 4035129463, 4035129479, 4035129482, 4035129483, 4035129484, 4035129497, 4035129522, 4035129523, 4035129531, 4035129533, 4035129536, 4035129546, 4035129547, 4035129556, 4035129559, 4035129560, 4035129583, 4035129584, 4035129591, 4035129594, 4035129598, 4035129601, 4035129610, 4035129622, 4035129632, 4035129646, 4035129657, 4035129665, 4035129671, 4035129683, 4035129698, 4035129706, 4035129707, 4035129716, 4035129729, 4035129905, 4035129925, 4035129931, 4035129932, 4035129939, 4035129979, 4035129994, 4035129995, 4035130008, 4035130012, 4035130025, 4035130052, 4035130057, 4035130078, 4035130079, 4035130096, 4035130100, 4035130103, 4035130118, 4035130124, 4035130128, 4035130131, 4035130139, 4035130140, 4035130154, 4035130193, 4035130200, 4035130214, 4035130224, 4035130225, 4035130245, 4035130246, 4035130251, 4035130261, 4035130283, 4035130286, 4035130292, 4035130294, 4035130307, 4035130315, 4035130321, 4035130322, 4035130324, 4035130341, 4035130353, 4035130368, 4035130371, 4035130393, 4035130394, 4035130396, 4035130423, 4035130430, 4035130434, 4035130449, 4035130457, 4035130464, 4035130472, 4035130473, 4035130482, 4035130485, 4035130488, 4035130503, 4035130505, 4035130530, 4035130536, 4035130560, 4035130564, 4035130573, 4035130591, 4035130607, 4035130608, 4035130611, 4035130623, 4035130630, 4035130631, 4035130634, 4035130639, 4035130645, 4035130648, 4035130651, 4035130655, 4035130671, 4035130674, 4035130675, 4035130684, 4035130716, 4035130720, 4035130724, 4035130734, 4035130736, 4035130738, 4035130743, 4035130744, 4035130746, 4035130753, 4035130756, 4035130763, 4035130766, 4035130803, 4035130813, 4035130814, 4035130820, 4035130868, 4035130874, 4035130886, 4035130897, 4035130898, 4035130930, 4035130937, 4035130957, 4035130985, 4035131010, 4035131047, 4035131050, 4035131070, 4035131082, 4035131086, 4035131097, 4035131102, 4035131106, 4035131109, 4035131113, 4035131122, 4035131143, 4035131158, 4035131160, 4035131173, 4035131174, 4035131194, 4035131197, 4035131198, 4035131207, 4035131220, 4035131230, 4035131231, 4035131232, 4035131233, 4035131261, 4035131276, 4035131286, 4035131299, 4035131310, 4035131316, 4035131340, 4035131347, 4035131356, 4035131369, 4035131381, 4035131385, 4035131387, 4035131392, 4035131399, 4035131410, 4035131411, 4035131412, 4035131416, 4035131421, 4035131441, 4035131445, 4035131451, 4035131455, 4035131458, 4035131464, 4035131474, 4035131488, 4035131528, 4035131533, 4035131535, 4035131540, 4035131546, 4035131549, 4035131552, 4035131560, 4035131572, 4035131578, 4035131602, 4035131603, 4035131610, 4035131618, 4035131619, 4035131631, 4035131636, 4035131646, 4035131648, 4035131661, 4035131669, 4035131670, 4035131678, 4035131683, 4035131684, 4035131709, 4035131714, 4035131721, 4035131726, 4035131728, 4035131751, 4035131755, 4035131782, 4035131799, 4035131800, 4035131804, 4035131806, 4035131842, 4035131845, 4035131862, 4035131866, 4035131872, 4035131873, 4035131882, 4035131886, 4035131887, 4035131897, 4035131905, 4035131907, 4035131932, 4035131941, 4035131949, 4035131955, 4035131965, 4035131973, 4035131979, 4035131992, 4035132000, 4035132005, 4035132024, 4035132026, 4035132033, 4035132036, 4035132040, 4035132046, 4035132053, 4035132067, 4035132070, 4035132071, 4035132078, 4035132082, 4035132083, 4035132089, 4035132093, 4035132097, 4035132114, 4035132115, 4035132122, 4035132129, 4035132130, 4035132136, 4035132147, 4035132150, 4035132151, 4035132160, 4035132162, 4035132167, 4035132168, 4035132208, 4035132221, 4035132223, 4035132225, 4035132233, 4035132237, 4035132242, 4035132246, 4035132264, 4035132270, 4035132278, 4035132298, 4035132308, 4035132327, 4035132361, 4035132372, 4035132379, 4035132390, 4035132393, 4035132395, 4035132405, 4035132410, 4035132428, 4035132432, 4035132437, 4035132444, 4035132448, 4035132459, 4035132472, 4035132489, 4035132501, 4035132505, 4035132506, 4035132515, 4035132522, 4035132536, 4035132540, 4035132557, 4035132574, 4035132601, 4035132628, 4035132632, 4035132634, 4035132635, 4035132643, 4035132654, 4035132657, 4035132682, 4035132683, 4035132692, 4035132726, 4035132749, 4035132751, 4035132758, 4035132763, 4035132775, 4035132779, 4035132788, 4035132808, 4035132821, 4035132822, 4035132831, 4035132853, 4035132887, 4035132894, 4035132905, 4035132907, 4035132918, 4035132921, 4035132924, 4035132927, 4035132931, 4035132937, 4035132951, 4035132958, 4035132959, 4035132976, 4035132980, 4035132981, 4035132986, 4035133000, 4035133017, 4035133024, 4035133038, 4035133045, 4035133046, 4035133059, 4035133085, 4035133089, 4035133092, 4035133104, 4035133132, 4035133143, 4035133169, 4035133196, 4035133201, 4035133241, 4035133248, 4035133256, 4035133264, 4035133272, 4035133285, 4035133306, 4035133307, 4035133320, 4035133329, 4035133345, 4035133353, 4035133358, 4035133361, 4035133365, 4035133366, 4035133396, 4035133404, 4035133406, 4035133427, 4035133432, 4035133440, 4035133441, 4035133444, 4035133446, 4035133447, 4035133451, 4035133467, 4035133486, 4035133487, 4035133500, 4035133504, 4035133506, 4035133514, 4035133541, 4035133558, 4035133565, 4035133581, 4035133587, 4035133610, 4035133644, 4035133650, 4035133652, 4035133657, 4035133659, 4035133660, 4035133688, 4035133695, 4035133702, 4035133704, 4035133709, 4035133712, 4035133716, 4035133740, 4035133758, 4035133763, 4035133782, 4035133795, 4035133804, 4035133830, 4035133835, 4035133840, 4035133842, 4035133845, 4035133862, 4035133884, 4035133907, 4035133914, 4035133923, 4035133927, 4035133928, 4035133930, 4035133946, 4035133953, 4035133963, 4035133970, 4035133981, 4035133987, 4035133992, 4035134013, 4035134018, 4035134030, 4035134046, 4035134053, 4035134060, 4035134069, 4035134076, 4035134079, 4035134114, 4035134125, 4035134132, 4035134138, 4035134148, 4035134149, 4035134154, 4035134155, 4035134169, 4035134195, 4035134207, 4035134218, 4035134247, 4035134248, 4035134260, 4035134263, 4035134273, 4035134294, 4035134302, 4035134303, 4035134306, 4035134316, 4035134318, 4035134328, 4035134345, 4035134350, 4035134352, 4035134375, 4035134382, 4035134393, 4035134400, 4035134415, 4035134417, 4035134430, 4035134431, 4035134472, 4035134497, 4035134504, 4035134511, 4035134519, 4035134538, 4035134544, 4035134561, 4035134571, 4035134581, 4035134584, 4035134586, 4035134588, 4035134596, 4035134598, 4035134607, 4035134608, 4035134629, 4035134637, 4035134642, 4035134644, 4035134655, 4035134665, 4035134691, 4035134695, 4035134715, 4035134716, 4035134761, 4035134762, 4035134768, 4035134778, 4035134779, 4035134785, 4035134795, 4035134802, 4035134806, 4035134807, 4035134821, 4035134823, 4035134831, 4035134835, 4035134845, 4035134858, 4035134859, 4035134872, 4035134873, 4035134891, 4035134893, 4035134900, 4035134911, 4035134942, 4035134943, 4035134947, 4035134949, 4035134964, 4035134969, 4035134972, 4035134985, 4035134990, 4035134991, 4035134998, 4035135004, 4035135018, 4035135024, 4035135047, 4035135051, 4035135055, 4035135057, 4035135081, 4035135089, 4035135093, 4035135098, 4035135102, 4035135103, 4035135105, 4035135106, 4035135112, 4035135123, 4035135128, 4035135129, 4035135157, 4035135173, 4035135194, 4035135207, 4035135235, 4035135240, 4035135253, 4035135255, 4035135277, 4035135309, 4035135310, 4035135312, 4035135315, 4035135322, 4035135340, 4035135358, 4035135359, 4035135386, 4035135390, 4035135396, 4035135397, 4035135411, 4035135426, 4035135434, 4035135446, 4035135450, 4035135455, 4035135462, 4035135465, 4035135469, 4035135473, 4035135500, 4035135545, 4035135552, 4035135563, 4035135567, 4035135568, 4035135578, 4035135596, 4035135609, 4035135615, 4035135616, 4035135619, 4035135634, 4035135638, 4035135646, 4035135661, 4035135673, 4035135678, 4035135681, 4035135682, 4035135689, 4035135702, 4035135721, 4035135728, 4035135730, 4035135740, 4035135746, 4035135748, 4035135749, 4035135750, 4035135751, 4035135760, 4035135775, 4035135811, 4035135848, 4035135855, 4035135874, 4035135880, 4035135881, 4035135884, 4035135889, 4035135894, 4035135897, 4035135898, 4035135923, 4035135924, 4035135927, 4035135928, 4035135930, 4035135937, 4035135964, 4035135965, 4035135972, 4035135986, 4035135989, 4035136007, 4035136027, 4035136054, 4035136058, 4035136060, 4035136068, 4035136076, 4035136086, 4035136111, 4035136124, 4035136128, 4035136136, 4035136142, 4035136151, 4035136160, 4035136166, 4035136199, 4035136209, 4035136210, 4035136217, 4035136238, 4035136246, 4035136248, 4035136252, 4035136264, 4035136266, 4035136270, 4035136273, 4035136293, 4035136301, 4035136320, 4035136323, 4035136325, 4035136341, 4035136342, 4035136345, 4035136349, 4035136371, 4035136403, 4035136417, 4035136430, 4035136432, 4035136446, 4035136458, 4035136474, 4035136492, 4035136503, 4035136518, 4035136523, 4035136526, 4035136527, 4035136539, 4035136560, 4035136563, 4035136573, 4035136577, 4035136579, 4035136584, 4035136585, 4035136604, 4035136610, 4035136620, 4035136621, 4035136628, 4035136633, 4035136648, 4035136664, 4035136674, 4035136715, 4035136723, 4035136736, 4035136755, 4035136772, 4035136792, 4035136801, 4035136816, 4035136826, 4035136834, 4035136836, 4035136849, 4035136895, 4035136906, 4035136913, 4035136921, 4035136922, 4035136924, 4035136926, 4035136935, 4035136954, 4035136966, 4035136985, 4035136986, 4035136995, 4035137034, 4035137056, 4035137057, 4035137065, 4035137083, 4035137086, 4035137107, 4035137118, 4035137129, 4035137135, 4035137137, 4035137138, 4035137158, 4035137161, 4035137167, 4035137179, 4035137197, 4035137198, 4035137243, 4035137247, 4035137251, 4035137291, 4035137293, 4035137294, 4035137297, 4035137300, 4035137306, 4035137308, 4035137321, 4035137330, 4035137343, 4035137353, 4035137368, 4035137369, 4035137373, 4035137387, 4035137388, 4035137389, 4035137395, 4035137411, 4035137423, 4035137434, 4035137438, 4035137452, 4035137457, 4035137475, 4035137488, 4035137491, 4035137494, 4035137499, 4035137511, 4035137525, 4035137542, 4035137570, 4035137578, 4035137588, 4035137619, 4035137630, 4035137643, 4035137650, 4035137666, 4035137668, 4035137672, 4035137673, 4035137683, 4035137699, 4035137739, 4035137748, 4035137766, 4035137775, 4035137783, 4035137784, 4035137808, 4035137810, 4035137820, 4035137822, 4035137830, 4035137834, 4035137835, 4035137848, 4035137866, 4035137880, 4035137885, 4035137887, 4035137899, 4035137904, 4035137905, 4035137924, 4035137936, 4035137951, 4035137969, 4035137972, 4035137975, 4035137980, 4035137982, 4035137985, 4035137992, 4035137994, 4035137996, 4035137997, 4035138016, 4035138024, 4035138030, 4035138035, 4035138038, 4035138042, 4035138059, 4035138063, 4035138070, 4035138071, 4035138074, 4035138079, 4035138080, 4035138083, 4035138084, 4035138087, 4035138090, 4035138100, 4035138104, 4035138110, 4035138111, 4035138126, 4035138128, 4035138151, 4035138154, 4035138171, 4035138181, 4035138190, 4035138215, 4035138232, 4035138239, 4035138247, 4035138272, 4035138295, 4035138306, 4035138308, 4035138310, 4035138320, 4035138331, 4035138350, 4035138356, 4035138385, 4035138391, 4035138392, 4035138408, 4035138414, 4035138418, 4035138420, 4035138432, 4035138434, 4035138446, 4035138450, 4035138461, 4035138477, 4035138501, 4035138504, 4035138505, 4035138506, 4035138513, 4035138545, 4035138579, 4035138581, 4035138606, 4035138609, 4035138617, 4035138618, 4035138620, 4035138624, 4035138645, 4035138672, 4035138676, 4035138681, 4035138692, 4035138693, 4035138694, 4035138696, 4035138716, 4035138721, 4035138723, 4035138740, 4035138744, 4035138747, 4035138751, 4035138752, 4035138754, 4035138756, 4035138786, 4035138795, 4035138823, 4035138825, 4035138847, 4035138854, 4035138859, 4035138863, 4035138874, 4035138879, 4035138880, 4035138887, 4035138897, 4035138898, 4035138900, 4035138903, 4035138914, 4035138915, 4035138920, 4035138940, 4035138954, 4035138963, 4035138977, 4035138997, 4035139004, 4035139024, 4035139028, 4035139054, 4035139055, 4035139074, 4035139075, 4035139088, 4035139093, 4035139107, 4035139121, 4035139143, 4035139146, 4035139149, 4035139153, 4035139161, 4035139162, 4035139165, 4035139202, 4035139235, 4035139239, 4035139276, 4035139290, 4035139291, 4035139296, 4035139300, 4035139310, 4035139313, 4035139334, 4035139356, 4035139358, 4035139362, 4035139385, 4035139387, 4035139397, 4035139405, 4035139411, 4035139423, 4035139441, 4035139458, 4035139472, 4035139485, 4035139492, 4035139496, 4035139497, 4035139500, 4035139547, 4035139553, 4035139560, 4035139562, 4035139597, 4035139608, 4035139610, 4035139613, 4035139618, 4035139622, 4035139624, 4035139634, 4035139636, 4035139639, 4035139660, 4035139666, 4035139670, 4035139679, 4035139706, 4035139711, 4035139715, 4035139719, 4035139757, 4035139758, 4035139767, 4035139781, 4035139789, 4035139807, 4035139809, 4035139819, 4035139822, 4035139828, 4035139834, 4035139838, 4035139857, 4035139862, 4035139872, 4035139885, 4035139886, 4035139898, 4035139900, 4035139903, 4035139910, 4035139923, 4035139925, 4035139940, 4035139959, 4035139968, 4035139989, 4035139993, 4035140002, 4035140009, 4035140011, 4035140014, 4035140020, 4035140052, 4035140053, 4035140066, 4035140072, 4035140073, 4035140076, 4035140078, 4035140120, 4035140128, 4035140131, 4035140133, 4035140141, 4035140143, 4035140150, 4035140155, 4035140161, 4035140170, 4035140190, 4035140197, 4035140202, 4035140208, 4035140214, 4035140217, 4035140218, 4035140221, 4035140230, 4035140235, 4035140236, 4035140239, 4035140243, 4035140274, 4035140278, 4035140300, 4035140302, 4035140308, 4035140314, 4035140322, 4035140339, 4035140349, 4035140358, 4035140375, 4035140380, 4035140387, 4035140390, 4035140395, 4035140425, 4035140431, 4035140454, 4035140458, 4035140460, 4035140464, 4035140468, 4035140469, 4035140470, 4035140476, 4035140490, 4035140491, 4035140492, 4035140504, 4035140506, 4035140517, 4035140530, 4035140532, 4035140536, 4035140537, 4035140551, 4035140569, 4035140575, 4035140578, 4035140628, 4035140629, 4035140646, 4035140649, 4035140655, 4035140669, 4035140671, 4035140698, 4035140699, 4035140704, 4035140717, 4035140726, 4035140789, 4035140791, 4035140797, 4035140798, 4035140811, 4035140821, 4035140835, 4035140841, 4035140848, 4035140855, 4035140877, 4035140882, 4035140890, 4035140905, 4035140915, 4035140917, 4035140924, 4035140925, 4035140941, 4035140944, 4035140955, 4035140959, 4035140976, 4035140981, 4035141006, 4035141023, 4035141024, 4035141042, 4035141056, 4035141063, 4035141080, 4035141084, 4035141086, 4035141091, 4035141100, 4035141101, 4035141114, 4035141115, 4035141117, 4035141141, 4035141142, 4035141143, 4035141144, 4035141157, 4035141175, 4035141190, 4035141191, 4035141202, 4035141203, 4035141206, 4035141210, 4035141211, 4035141233, 4035141235, 4035141260, 4035141270, 4035141290, 4035141291, 4035141295, 4035141307, 4035141370, 4035141374, 4035141383, 4035141387, 4035141410, 4035141438, 4035141447, 4035141456, 4035141469, 4035141478, 4035141494, 4035141505, 4035141511, 4035141523, 4035141550, 4035141562, 4035141564, 4035141578, 4035141592, 4035141593, 4035141601, 4035141609, 4035141622, 4035141635, 4035141637, 4035141638, 4035141667, 4035141676, 4035141677, 4035141697, 4035141698, 4035141702, 4035141720, 4035141724, 4035141736, 4035141741, 4035141757, 4035141785, 4035141817, 4035141840, 4035141848, 4035141852, 4035141891, 4035141899, 4035141917, 4035141927, 4035141936, 4035141938, 4035141945, 4035141949, 4035141957, 4035141961, 4035141965, 4035141985, 4035141989, 4035141990, 4035141994, 4035141998, 4035142009, 4035142020, 4035142023, 4035142028, 4035142037, 4035142040, 4035142060, 4035142067, 4035142070, 4035142072, 4035142085, 4035142087, 4035142103, 4035142104, 4035142110, 4035142145, 4035142147, 4035142155, 4035142166, 4035142172, 4035142177, 4035142178, 4035142207, 4035142234, 4035142265, 4035142274, 4035142295, 4035142298, 4035142320, 4035142323, 4035142328, 4035142329, 4035142337, 4035142346, 4035142347, 4035142348, 4035142355, 4035142378, 4035142391, 4035142406, 4035142414, 4035142429, 4035142439, 4035142451, 4035142463, 4035142486, 4035142494, 4035142495, 4035142505, 4035142513, 4035142520, 4035142559, 4035142582, 4035142591, 4035142596, 4035142627, 4035142636, 4035142649, 4035142662, 4035142672, 4035142674, 4035142683, 4035142688, 4035142702, 4035142704, 4035142713, 4035142721, 4035142728, 4035142734, 4035142752, 4035142761, 4035142770, 4035142775, 4035142778, 4035142780, 4035142784, 4035142793, 4035142799, 4035142802, 4035142817, 4035142826, 4035142833, 4035142846, 4035142852, 4035142858, 4035142864, 4035142875, 4035142882, 4035142913, 4035142924, 4035142931, 4035142936, 4035142947, 4035142951, 4035142959, 4035142963, 4035142971, 4035142974, 4035142975, 4035142976, 4035142980, 4035142982, 4035142998, 4035143009, 4035143037, 4035143052, 4035143060, 4035143083, 4035143084, 4035143088, 4035143100, 4035143122, 4035143132, 4035143134, 4035143135, 4035143139, 4035143152, 4035143163, 4035143170, 4035143171, 4035143188, 4035143194, 4035143202, 4035143235, 4035143261, 4035143272, 4035143273, 4035143282, 4035143284, 4035143292, 4035143298, 4035143300, 4035143321, 4035143330, 4035143336, 4035143340, 4035143347, 4035143353, 4035143357, 4035143372, 4035143386, 4035143389, 4035143390, 4035143403, 4035143414, 4035143416, 4035143418, 4035143419, 4035143441, 4035143442, 4035143452, 4035143475, 4035143476, 4035143482, 4035143499, 4035143500, 4035143505, 4035143523, 4035143538, 4035143540, 4035143543, 4035143553, 4035143554, 4035143560, 4035143567, 4035143573, 4035143574, 4035143576, 4035143578, 4035143583, 4035143611, 4035143615, 4035143618, 4035143620, 4035143621, 4035143632, 4035143644, 4035143649, 4035143650, 4035143654, 4035143659, 4035143661, 4035143671, 4035143672, 4035143676, 4035143683, 4035143684, 4035143685, 4035143696, 4035143709, 4035143721, 4035143729, 4035143757, 4035143759, 4035143776, 4035143823, 4035143824, 4035143828, 4035143830, 4035143846, 4035143853, 4035143883, 4035143888, 4035143896, 4035143899, 4035143959, 4035143960, 4035143969, 4035143987, 4035144004, 4035144005, 4035144016, 4035144037, 4035144038, 4035144039, 4035144041, 4035144049, 4035144071, 4035144084, 4035144103, 4035144104, 4035144105, 4035144119, 4035144127, 4035144137, 4035144141, 4035144149, 4035144153, 4035144157, 4035144164, 4035144167, 4035144169, 4035144177, 4035144185, 4035144190, 4035144214, 4035144216, 4035144218, 4035144225, 4035144227, 4035144228, 4035144232, 4035144233, 4035144242, 4035144248, 4035144249, 4035144259, 4035144267, 4035144268, 4035144273, 4035144275, 4035144276, 4035144290, 4035144301, 4035144305, 4035144327, 4035144330, 4035144334, 4035144344, 4035144357, 4035144358, 4035144384, 4035144390, 4035144396, 4035144397, 4035144411, 4035144429, 4035144432, 4035144442, 4035144445, 4035144458, 4035144464, 4035144475, 4035144479, 4035144485, 4035144486, 4035144510, 4035144514, 4035144536, 4035144538, 4035144539, 4035144542, 4035144543, 4035144580, 4035144584, 4035144588, 4035144594, 4035144614, 4035144617, 4035144633, 4035144634, 4035144663, 4035144681, 4035144701, 4035144709, 4035144712, 4035144713, 4035144723, 4035144758, 4035144773, 4035144783, 4035144800, 4035144820, 4035144826, 4035144836, 4035144840, 4035144873, 4035144890, 4035144903, 4035144913, 4035144914, 4035144920, 4035144921, 4035144924, 4035144941, 4035144963, 4035144997, 4035145000, 4035145014, 4035145016, 4035145029, 4035145039, 4035145058, 4035145067, 4035145079, 4035145084, 4035145111, 4035145120, 4035145130, 4035145131, 4035145133, 4035145139, 4035145170, 4035145184, 4035145202, 4035145219, 4035145229, 4035145236, 4035145239, 4035145251, 4035145274, 4035145301, 4035145302, 4035145303, 4035145351, 4035145354, 4035145360, 4035145376, 4035145387, 4035145390, 4035145394, 4035145398, 4035145399, 4035145409, 4035145412, 4035145427, 4035145437, 4035145440, 4035145447, 4035145471, 4035145477, 4035145478, 4035145480, 4035145491, 4035145493, 4035145498, 4035145501, 4035145526, 4035145533, 4035145546, 4035145555, 4035145559, 4035145560, 4035145563, 4035145569, 4035145574, 4035145575, 4035145596, 4035145609, 4035145613, 4035145618, 4035145623, 4035145660, 4035145683, 4035145690, 4035145696, 4035145699, 4035145709, 4035145736, 4035145751, 4035145756, 4035145767, 4035145770, 4035145773, 4035145776, 4035145794, 4035145800, 4035145816, 4035145826, 4035145829, 4035145845, 4035145846, 4035145864, 4035145868, 4035145876, 4035145882, 4035145918, 4035145933, 4035145941, 4035145952, 4035145965, 4035145982, 4035145985, 4035145988, 4035145995, 4035145997, 4035146017, 4035146038, 4035146057, 4035146062, 4035146090, 4035146096, 4035146098, 4035146105, 4035146118, 4035146126, 4035146128, 4035146146, 4035146187, 4035146194, 4035146217, 4035146222, 4035146245, 4035146270, 4035146271, 4035146284, 4035146296, 4035146307, 4035146325, 4035146326, 4035146338, 4035146370, 4035146381, 4035146390, 4035146392, 4035146397, 4035146408, 4035146409, 4035146418, 4035146442, 4035146449, 4035146468, 4035146469, 4035146485, 4035146496, 4035146500, 4035146507, 4035146529, 4035146530, 4035146533, 4035146555, 4035146558, 4035146559, 4035146574, 4035146580, 4035146587, 4035146588, 4035146604, 4035146643, 4035146662, 4035146664, 4035146689, 4035146702, 4035146725, 4035146743, 4035146746, 4035146749, 4035146759, 4035146776, 4035146836, 4035146840, 4035146844, 4035146872, 4035146875, 4035146883, 4035146884, 4035146892, 4035146898, 4035146899, 4035146908, 4035146913, 4035146918, 4035146927, 4035146940, 4035146945, 4035146952, 4035146972, 4035146973, 4035146998, 4035147001, 4035147004, 4035147007, 4035147016, 4035147027, 4035147032, 4035147045, 4035147063, 4035147086, 4035147091, 4035147097, 4035147126, 4035147163, 4035147177, 4035147192, 4035147200, 4035147224, 4035147225, 4035147232, 4035147237, 4035147243, 4035147244, 4035147245, 4035147289, 4035147293, 4035147296, 4035147304, 4035147317, 4035147337, 4035147346, 4035147349, 4035147351, 4035147369, 4035147375, 4035147380, 4035147392, 4035147395, 4035147414, 4035147420, 4035147423, 4035147425, 4035147430, 4035147431, 4035147440, 4035147447, 4035147469, 4035147478, 4035147494, 4035147501, 4035147509, 4035147518, 4035147528, 4035147529, 4035147544, 4035147547, 4035147554, 4035147555, 4035147563, 4035147565, 4035147566, 4035147569, 4035147580, 4035147581, 4035147589, 4035147617, 4035147624, 4035147638, 4035147667, 4035147670, 4035147681, 4035147684, 4035147685, 4035147698, 4035147701, 4035147711, 4035147713, 4035147748, 4035147756, 4035147757, 4035147760, 4035147777, 4035147810, 4035147840, 4035147841, 4035147864, 4035147877, 4035147881, 4035147885, 4035147888, 4035147911, 4035147912, 4035147914, 4035147920, 4035147925, 4035147930, 4035147933, 4035147939, 4035147959, 4035147963, 4035147977, 4035147981, 4035147994, 4035147996, 4035148003, 4035148017, 4035148042, 4035148060, 4035148069, 4035148092, 4035148097, 4035148116, 4035148117, 4035148127, 4035148129, 4035148131, 4035148136, 4035148137, 4035148142, 4035148143, 4035148147, 4035148156, 4035148161, 4035148162, 4035148165, 4035148169, 4035148180, 4035148186, 4035148194, 4035148199, 4035148211, 4035148220, 4035148224, 4035148234, 4035148248, 4035148249, 4035148253, 4035148261, 4035148263, 4035148264, 4035148269, 4035148274, 4035148277, 4035148285, 4035148286, 4035148293, 4035148308, 4035148310, 4035148311, 4035148313, 4035148322, 4035148341, 4035148344, 4035148358, 4035148369, 4035148374, 4035148377, 4035148378, 4035148382, 4035148395, 4035148397, 4035148405, 4035148421, 4035148436, 4035148461, 4035148475, 4035148496, 4035148535, 4035148538, 4035148553, 4035148568, 4035148570, 4035148575, 4035148588, 4035148597, 4035148599, 4035148603, 4035148611, 4035148618, 4035148622, 4035148638, 4035148646, 4035148658, 4035148661, 4035148687, 4035148695, 4035148701, 4035148706, 4035148711, 4035148713, 4035148721, 4035148730, 4035148763, 4035148764, 4035148767, 4035148785, 4035148813, 4035148816, 4035148819, 4035148822, 4035148823, 4035148827, 4035148833, 4035148840, 4035148841, 4035148845, 4035148851, 4035148867, 4035148883, 4035148890, 4035148891, 4035148910, 4035148919, 4035148930, 4035148948, 4035148986, 4035148987, 4035148996, 4035149003, 4035149018, 4035149019, 4035149021, 4035149047, 4035149050, 4035149061, 4035149068, 4035149069, 4035149071, 4035149077, 4035149079, 4035149084, 4035149096, 4035149097, 4035149098, 4035149103, 4035149119, 4035149131, 4035149154, 4035149159, 4035149163, 4035149172, 4035149176, 4035149194, 4035149197, 4035149203, 4035149204, 4035149220, 4035149231, 4035149232, 4035149244, 4035149250, 4035149270, 4035149274, 4035149276, 4035149282, 4035149302, 4035149312, 4035149332, 4035149350, 4035149356, 4035149372, 4035149387, 4035149392, 4035149399, 4035149421, 4035149427, 4035149433, 4035149462, 4035149467, 4035149470, 4035149475, 4035149482, 4035149503, 4035149515, 4035149519, 4035149520, 4035149528, 4035149567, 4035149581, 4035149587, 4035149589, 4035149595, 4035149612, 4035149627, 4035149643, 4035149644, 4035149651, 4035149662, 4035149709, 4035149723, 4035149726, 4035149731, 4035149733, 4035149744, 4035149745, 4035149746, 4035149776, 4035149791, 4035149809, 4035149811, 4035149835, 4035149837, 4035149844, 4035149854, 4035149860, 4035149865, 4035149870, 4035149874, 4035149894, 4035149908, 4035149913, 4035149916, 4035149940, 4035149958, 4035149959, 4035149980, 4035149988, 4035149993, 4035149998, 4035150001, 4035150006, 4035150014, 4035150015, 4035150018, 4035150076, 4035150088, 4035150102, 4035150105, 4035150107, 4035150115, 4035150128, 4035150135, 4035150153, 4035150166, 4035150170, 4035150171, 4035150174, 4035150178, 4035150179, 4035150206, 4035150234, 4035150237, 4035150247, 4035150253, 4035150257, 4035150258, 4035150261, 4035150270, 4035150278, 4035150293, 4035150305, 4035150327, 4035150344, 4035150347, 4035150355, 4035150365, 4035150369, 4035150380, 4035150395, 4035150401, 4035150402, 4035150420, 4035150444, 4035150475, 4035150476, 4035150491, 4035150497, 4035150501, 4035150521, 4035150524, 4035150528, 4035150530, 4035150556, 4035150561, 4035150583, 4035150588, 4035150593, 4035150603, 4035150613, 4035150614, 4035150620, 4035150622, 4035150629, 4035150630, 4035150650, 4035150657, 4035150662, 4035150663, 4035150666, 4035150667, 4035150669, 4035150673, 4035150675, 4035150707, 4035150717, 4035150739, 4035150754, 4035150762, 4035150788, 4035150821, 4035150822, 4035150823, 4035150829, 4035150832, 4035150859, 4035150860, 4035150888, 4035150902, 4035150917, 4035150943, 4035150952, 4035150968, 4035150975, 4035150981, 4035150982, 4035150988, 4035150989, 4035151022, 4035151023, 4035151024, 4035151038, 4035151077, 4035151082, 4035151106, 4035151123, 4035151130, 4035151191, 4035151195, 4035151196, 4035151202, 4035151212, 4035151214, 4035151223, 4035151224, 4035151228, 4035151232, 4035151252, 4035151277, 4035151278, 4035151290, 4035151302, 4035151312, 4035151331, 4035151342, 4035151344, 4035151345, 4035151346, 4035151351, 4035151352, 4035151355, 4035151361, 4035151367, 4035151378, 4035151385, 4035151390, 4035151394, 4035151413, 4035151423, 4035151430, 4035151432, 4035151445, 4035151450, 4035151453, 4035151455, 4035151478, 4035151481, 4035151488, 4035151489, 4035151504, 4035151505, 4035151509, 4035151522, 4035151531, 4035151536, 4035151539, 4035151547, 4035151562, 4035151566, 4035151574, 4035151579, 4035151588, 4035151591, 4035151603, 4035151605, 4035151606, 4035151613, 4035151639, 4035151647, 4035151650, 4035151655, 4035151662, 4035151674, 4035151677, 4035151681, 4035151685, 4035151688, 4035151692, 4035151708, 4035151728, 4035151737, 4035151756, 4035151776, 4035151785, 4035151790, 4035151795, 4035151804, 4035151824, 4035151825, 4035151838, 4035151862, 4035151863, 4035151917, 4035151924, 4035151928, 4035151931, 4035151933, 4035151958, 4035151978, 4035151979, 4035151997, 4035152020, 4035152026, 4035152030, 4035152031, 4035152043, 4035152076, 4035152077, 4035152095, 4035152097, 4035152098, 4035152108, 4035152115, 4035152119, 4035152122, 4035152127, 4035152129, 4035152133, 4035152138, 4035152139, 4035152141, 4035152145, 4035152152, 4035152165, 4035152170, 4035152171, 4035152175, 4035152195, 4035152207, 4035152209, 4035152224, 4035152230, 4035152233, 4035152238, 4035152242, 4035152249, 4035152265, 4035152268, 4035152272, 4035152293, 4035152294, 4035152295, 4035152301, 4035152312, 4035152322, 4035152324, 4035152328, 4035152333, 4035152337, 4035152349, 4035152391, 4035152392, 4035152394, 4035152416, 4035152430, 4035152454, 4035152458, 4035152468, 4035152472, 4035152503, 4035152506, 4035152507, 4035152508, 4035152523, 4035152528, 4035152529, 4035152531, 4035152532, 4035152533, 4035152538, 4035152544, 4035152552, 4035152553, 4035152554, 4035152561, 4035152564, 4035152565, 4035152577, 4035152583, 4035152584, 4035152596, 4035152603, 4035152605, 4035152614, 4035152622, 4035152625, 4035152637, 4035152639, 4035152650, 4035152666, 4035152667, 4035152671, 4035152677, 4035152679, 4035152681, 4035152682, 4035152693, 4035152702, 4035152703, 4035152707, 4035152715, 4035152728, 4035152733, 4035152741, 4035152746, 4035152749, 4035152765, 4035152786, 4035152804, 4035152810, 4035152825, 4035152840, 4035152843, 4035152869, 4035152883, 4035152889, 4035152899, 4035152921, 4035152929, 4035152955, 4035152962, 4035152963, 4035152964, 4035152987, 4035153013, 4035153027, 4035153038, 4035153050, 4035153084, 4035153085, 4035153089, 4035153092, 4035153098, 4035153116, 4035153129, 4035153134, 4035153145, 4035153148, 4035153149, 4035153151, 4035153184, 4035153192, 4035153201, 4035153203, 4035153208, 4035153212, 4035153225, 4035153244, 4035153245, 4035153246, 4035153247, 4035153255, 4035153272, 4035153308, 4035153321, 4035153343, 4035153349, 4035153352, 4035153364, 4035153372, 4035153377, 4035153379, 4035153398, 4035153402, 4035153428, 4035153430, 4035153445, 4035153446, 4035153458, 4035153464, 4035153473, 4035153480, 4035153483, 4035153484, 4035153492, 4035153495, 4035153513, 4035153518, 4035153526, 4035153551, 4035153580, 4035153585, 4035153586, 4035153600, 4035153607, 4035153614, 4035153624, 4035153654, 4035153657, 4035153665, 4035153676, 4035153677, 4035153692, 4035153699, 4035153724, 4035153730, 4035153739, 4035153756, 4035153759, 4035153760, 4035153773, 4035153774, 4035153777, 4035153782, 4035153783, 4035153795, 4035153820, 4035153821, 4035153842, 4035153843, 4035153844, 4035153857, 4035153860, 4035153863, 4035153869, 4035153880, 4035153884, 4035153885, 4035153910, 4035153911, 4035153912, 4035153925, 4035153935, 4035153936, 4035153938, 4035153940, 4035153948, 4035153950, 4035153953, 4035153955, 4035153994, 4035153996, 4035153997, 4035154000, 4035154016, 4035154031, 4035154054, 4035154061, 4035154072, 4035154081, 4035154083, 4035154090, 4035154092, 4035154093, 4035154117, 4035154128, 4035154129, 4035154137, 4035154138, 4035154147, 4035154149, 4035154152, 4035154153, 4035154158, 4035154159, 4035154162, 4035154163, 4035154210, 4035154220, 4035154221, 4035154223, 4035154231, 4035154234, 4035154238, 4035154247, 4035154250, 4035154265, 4035154270, 4035154294, 4035154296, 4035154300, 4035154305, 4035154307, 4035154318, 4035154331, 4035154333, 4035154335, 4035154349, 4035154370, 4035154383, 4035154387, 4035154416, 4035154420, 4035154421, 4035154425, 4035154434, 4035154459, 4035154461, 4035154482, 4035154484, 4035154507, 4035154509, 4035154511, 4035154514, 4035154534, 4035154544, 4035154546, 4035154548, 4035154560, 4035154579, 4035154624, 4035154627, 4035154630, 4035154644, 4035154651, 4035154655, 4035154665, 4035154666, 4035154680, 4035154681, 4035154683, 4035154689, 4035154691, 4035154704, 4035154724, 4035154731, 4035154760, 4035154762, 4035154784, 4035154794, 4035154836, 4035154837, 4035154838, 4035154850, 4035154853, 4035154859, 4035154878, 4035154893, 4035154899, 4035154907, 4035154917, 4035154940, 4035154941, 4035154958, 4035154967, 4035154970, 4035154979, 4035154989, 4035154995, 4035155007, 4035155011, 4035155023, 4035155032, 4035155044, 4035155047, 4035155053, 4035155068, 4035155089, 4035155099, 4035155115, 4035155119, 4035155134, 4035155146, 4035155154, 4035155160, 4035155162, 4035155163, 4035155176, 4035155181, 4035155191, 4035155247, 4035155255, 4035155256, 4035155260, 4035155287, 4035155290, 4035155297, 4035155328, 4035155330, 4035155333, 4035155377, 4035155395, 4035155396, 4035155397, 4035155398, 4035155418, 4035155427, 4035155440, 4035155446, 4035155449, 4035155456, 4035155481, 4035155482, 4035155502, 4035155506, 4035155508, 4035155539, 4035155540, 4035155548, 4035155554, 4035155565, 4035155566, 4035155567, 4035155578, 4035155583, 4035155592, 4035155593, 4035155622, 4035155632, 4035155635, 4035155671, 4035155673, 4035155722, 4035155726, 4035155735, 4035155739, 4035155744, 4035155748, 4035155749, 4035155758, 4035155763, 4035155768, 4035155783, 4035155820, 4035155870, 4035155880, 4035155884, 4035155896, 4035155898, 4035155899, 4035155913, 4035155916, 4035155947, 4035155958, 4035155959, 4035155972, 4035155976, 4035155979, 4035155986, 4035155987, 4035156000, 4035156002, 4035156017, 4035156025, 4035156053, 4035156082, 4035156088, 4035156097, 4035156117, 4035156125, 4035156132, 4035156139, 4035156144, 4035156155, 4035156158, 4035156171, 4035156173, 4035156188, 4035156191, 4035156192, 4035156193, 4035156194, 4035156206, 4035156237, 4035156240, 4035156268, 4035156273, 4035156281, 4035156283, 4035156297, 4035156299, 4035156307, 4035156323, 4035156330, 4035156331, 4035156344, 4035156353, 4035156356, 4035156357, 4035156374, 4035156377, 4035156387, 4035156418, 4035156425, 4035156426, 4035156429, 4035156434, 4035156448, 4035156453, 4035156458, 4035156460, 4035156486, 4035156488, 4035156492, 4035156493, 4035156502, 4035156528, 4035156553, 4035156598, 4035156601, 4035156607, 4035156617, 4035156632, 4035156645, 4035156667, 4035156684, 4035156700, 4035156720, 4035156721, 4035156729, 4035156735, 4035156737, 4035156744, 4035156758, 4035156762, 4035156774, 4035156785, 4035156812, 4035156819, 4035156820, 4035156824, 4035156829, 4035156840, 4035156846, 4035156847, 4035156850, 4035156851, 4035156854, 4035156881, 4035156894, 4035156903, 4035156929, 4035156932, 4035156945, 4035156947, 4035156948, 4035156949, 4035156972, 4035156975, 4035156985, 4035156995, 4035157004, 4035157008, 4035157009, 4035157016, 4035157033, 4035157042, 4035157059, 4035157081, 4035157085, 4035157096, 4035157113, 4035157118, 4035157120, 4035157126, 4035157152, 4035157161, 4035157169, 4035157170, 4035157191, 4035157211, 4035157221, 4035157253, 4035157254, 4035157256, 4035157272, 4035157274, 4035157281, 4035157286, 4035157292, 4035157306, 4035157313, 4035157315, 4035157339, 4035157360, 4035157361, 4035157372, 4035157377, 4035157379, 4035157396, 4035157398, 4035157411, 4035157412, 4035157415, 4035157441, 4035157444, 4035157453, 4035157458, 4035157466, 4035157477, 4035157493, 4035157508, 4035157517, 4035157524, 4035157529, 4035157538, 4035157556, 4035157570, 4035157607, 4035157613, 4035157615, 4035157630, 4035157634, 4035157645, 4035157646, 4035157658, 4035157667, 4035157677, 4035157686, 4035157691, 4035157692, 4035157695, 4035157703, 4035157726, 4035157741, 4035157792, 4035157809, 4035157829, 4035157855, 4035157882, 4035157890, 4035157900, 4035157918, 4035157922, 4035157924, 4035157942, 4035157958, 4035157964, 4035157966, 4035157972, 4035157981, 4035157991, 4035158021, 4035158035, 4035158046, 4035158047, 4035158092, 4035158098, 4035158099, 4035158102, 4035158113, 4035158120, 4035158126, 4035158140, 4035158155, 4035158166, 4035158178, 4035158189, 4035158204, 4035158208, 4035158209, 4035158220, 4035158236, 4035158240, 4035158249, 4035158255, 4035158256, 4035158258, 4035158261, 4035158264, 4035158269, 4035158284, 4035158300, 4035158303, 4035158335, 4035158355, 4035158384, 4035158395, 4035158412, 4035158428, 4035158435, 4035158437, 4035158453, 4035158469, 4035158471, 4035158478, 4035158484, 4035158488, 4035158493, 4035158499, 4035158507, 4035158513, 4035158530, 4035158531, 4035158537, 4035158562, 4035158563, 4035158566, 4035158582, 4035158593, 4035158598, 4035158605, 4035158606, 4035158617, 4035158620, 4035158625, 4035158637, 4035158647, 4035158669, 4035158674, 4035158688, 4035158691, 4035158693, 4035158704, 4035158705, 4035158708, 4035158719, 4035158727, 4035158735, 4035158762, 4035158769, 4035158771, 4035158809, 4035158827, 4035158828, 4035158881, 4035158883, 4035158912, 4035158960, 4035158964, 4035158966, 4035158970, 4035158976, 4035158989, 4035158997, 4035159000, 4035159001, 4035159011, 4035159016, 4035159022, 4035159023, 4035159048, 4035159061, 4035159079, 4035159080, 4035159094, 4035159138, 4035159142, 4035159146, 4035159147, 4035159164, 4035159166, 4035159192, 4035159197, 4035159203, 4035159218, 4035159223, 4035159225, 4035159245, 4035159247, 4035159248, 4035159255, 4035159268, 4035159269, 4035159272, 4035159284, 4035159286, 4035159305, 4035159306, 4035159309, 4035159316, 4035159329, 4035159334, 4035159342, 4035159343, 4035159347, 4035159349, 4035159363, 4035159374, 4035159394, 4035159424, 4035159434, 4035159443, 4035159451, 4035159454, 4035159468, 4035159494, 4035159501, 4035159518, 4035159526, 4035159529, 4035159557, 4035159570, 4035159572, 4035159595, 4035159596, 4035159598, 4035159604, 4035159620, 4035159626, 4035159634, 4035159652, 4035159669, 4035159676, 4035159677, 4035159680, 4035159683, 4035159688, 4035159705, 4035159711, 4035159742, 4035159749, 4035159750, 4035159760, 4035159772, 4035159799, 4035159810, 4035159824, 4035159827, 4035159835, 4035159843, 4035159854, 4035159866, 4035159867, 4035159874, 4035159892, 4035159901, 4035159924, 4035159933, 4035159956, 4035159967, 4035159968, 4035159971, 4035159988, 4035160003, 4035160014, 4035160016, 4035160019, 4035160023, 4035160072, 4035160084, 4035160129, 4035160132, 4035160143, 4035160208, 4035160233, 4035160241, 4035160249, 4035160267, 4035160269, 4035160272, 4035160280, 4035160285, 4035160294, 4035160298, 4035160346, 4035160362, 4035160366, 4035160367, 4035160369, 4035160374, 4035160379, 4035160392, 4035160407, 4035160409, 4035160411, 4035160421, 4035160435, 4035160438, 4035160444, 4035160473, 4035160478, 4035160481, 4035160488, 4035160489, 4035160500, 4035160503, 4035160507, 4035160513, 4035160520, 4035160525, 4035160527, 4035160529, 4035160530, 4035160538, 4035160546, 4035160548, 4035160549, 4035160550, 4035160568, 4035160590, 4035160607, 4035160611, 4035160650, 4035160656, 4035160657, 4035160664, 4035160675, 4035160684, 4035160686, 4035160687, 4035160690, 4035160698, 4035160710, 4035160736, 4035160792, 4035160793, 4035160828, 4035160829, 4035160832, 4035160843, 4035160858, 4035160863, 4035160867, 4035160874, 4035160884, 4035160888, 4035160890, 4035160892, 4035160897, 4035160911, 4035160917, 4035160928, 4035160932, 4035160939, 4035160943, 4035160944, 4035160956, 4035160957, 4035160964, 4035160966, 4035160987, 4035160990, 4035160991, 4035161001, 4035161014, 4035161030, 4035161031, 4035161044, 4035161074, 4035161079, 4035161104, 4035161107, 4035161111, 4035161112, 4035161131, 4035161132, 4035161135, 4035161139, 4035161156, 4035161160, 4035161163, 4035161168, 4035161210, 4035161211, 4035161217, 4035161224, 4035161230, 4035161244, 4035161253, 4035161260, 4035161275, 4035161279, 4035161294, 4035161295, 4035161310, 4035161316, 4035161343, 4035161352, 4035161357, 4035161359, 4035161360, 4035161388, 4035161403, 4035161419, 4035161449, 4035161474, 4035161478, 4035161480, 4035161488, 4035161490, 4035161491, 4035161497, 4035161499, 4035161502, 4035161513, 4035161514, 4035161527, 4035161546, 4035161570, 4035161591, 4035161600, 4035161602, 4035161621, 4035161627, 4035161645, 4035161650, 4035161661, 4035161668, 4035161679, 4035161689, 4035161700, 4035161701, 4035161717, 4035161743, 4035161756, 4035161761, 4035161775, 4035161776, 4035161779, 4035161780, 4035161781, 4035161787, 4035161829, 4035161830, 4035161851, 4035161856, 4035161888, 4035161889, 4035161899, 4035161900, 4035161901, 4035161902, 4035161906, 4035161911, 4035161912, 4035161926, 4035161927, 4035161934, 4035161959, 4035161976, 4035161993, 4035162014, 4035162020, 4035162048, 4035162050, 4035162056, 4035162075, 4035162080, 4035162094, 4035162151, 4035162164, 4035162192, 4035162217, 4035162219, 4035162220, 4035162222, 4035162223, 4035162224, 4035162231, 4035162246, 4035162252, 4035162256, 4035162284, 4035162287, 4035162327, 4035162332, 4035162355, 4035162356, 4035162380, 4035162390, 4035162393, 4035162394, 4035162403, 4035162405, 4035162417, 4035162427, 4035162429, 4035162433, 4035162435, 4035162445, 4035162476, 4035162506, 4035162508, 4035162529, 4035162534, 4035162544, 4035162562, 4035162573, 4035162579, 4035162585, 4035162586, 4035162608, 4035162614, 4035162617, 4035162636, 4035162654, 4035162655, 4035162673, 4035162689, 4035162694, 4035162703, 4035162710, 4035162714, 4035162715, 4035162716, 4035162720, 4035162729, 4035162758, 4035162785, 4035162790, 4035162808, 4035162815, 4035162820, 4035162822, 4035162841, 4035162847, 4035162863, 4035162864, 4035162872, 4035162899, 4035162903, 4035162904, 4035162907, 4035162912, 4035162916, 4035162926, 4035162930, 4035162933, 4035162939, 4035162948, 4035162956, 4035162968, 4035162980, 4035162985, 4035163001, 4035163007, 4035163009, 4035163022, 4035163033, 4035163055, 4035163064, 4035163065, 4035163066, 4035163069, 4035163076, 4035163077, 4035163096, 4035163101, 4035163123, 4035163141, 4035163144, 4035163145, 4035163151, 4035163156, 4035163192, 4035163198, 4035163203, 4035163228, 4035163229, 4035163242, 4035163249, 4035163254, 4035163292, 4035163296, 4035163300, 4035163327, 4035163342, 4035163346, 4035163349, 4035163352, 4035163354, 4035163363, 4035163368, 4035163381, 4035163390, 4035163418, 4035163431, 4035163432, 4035163452, 4035163454, 4035163460, 4035163469, 4035163496, 4035163502, 4035163510, 4035163516, 4035163563, 4035163564, 4035163568, 4035163569, 4035163594, 4035163596, 4035163619, 4035163620, 4035163636, 4035163646, 4035163648, 4035163649, 4035163658, 4035163681, 4035163685, 4035163689, 4035163715, 4035163716, 4035163726, 4035163732, 4035163737, 4035163742, 4035163764, 4035163785, 4035163787, 4035163792, 4035163797, 4035163802, 4035163809, 4035163821, 4035163842, 4035163855, 4035163859, 4035163861, 4035163866, 4035163877, 4035163884, 4035163900, 4035163908, 4035163918, 4035163924, 4035163937, 4035163938, 4035163939, 4035163942, 4035163950, 4035163981, 4035163992, 4035163999, 4035164004, 4035164006, 4035164029, 4035164031, 4035164054, 4035164059, 4035164064, 4035164067, 4035164086, 4035164115, 4035164119, 4035164126, 4035164149, 4035164159, 4035164182, 4035164188, 4035164196, 4035164208, 4035164220, 4035164222, 4035164226, 4035164235, 4035164246, 4035164247, 4035164262, 4035164273, 4035164293, 4035164297, 4035164307, 4035164319, 4035164324, 4035164325, 4035164327, 4035164336, 4035164345, 4035164353, 4035164357, 4035164362, 4035164393, 4035164399, 4035164403, 4035164411, 4035164413, 4035164426, 4035164434, 4035164469, 4035164473, 4035164480, 4035164494, 4035164511, 4035164512, 4035164518, 4035164519, 4035164520, 4035164523, 4035164529, 4035164545, 4035164547, 4035164551, 4035164555, 4035164558, 4035164562, 4035164570, 4035164574, 4035164586, 4035164602, 4035164603, 4035164617, 4035164625, 4035164638, 4035164648, 4035164652, 4035164657, 4035164665, 4035164669, 4035164682, 4035164693, 4035164697, 4035164698, 4035164715, 4035164720, 4035164735, 4035164736, 4035164737, 4035164741, 4035164743, 4035164748, 4035164752, 4035164762, 4035164766, 4035164769, 4035164779, 4035164787, 4035164811, 4035164821, 4035164848, 4035164869, 4035164877, 4035164895, 4035164925, 4035164955, 4035164965, 4035164972, 4035164974, 4035164975, 4035164977, 4035164989, 4035164991, 4035164996, 4035165018, 4035165027, 4035165059, 4035165087, 4035165098, 4035165117, 4035165118, 4035165120, 4035165123, 4035165135, 4035165155, 4035165165, 4035165168, 4035165178, 4035165179, 4035165184, 4035165222, 4035165228, 4035165246, 4035165253, 4035165254, 4035165255, 4035165270, 4035165277, 4035165281, 4035165284, 4035165305, 4035165314, 4035165315, 4035165317, 4035165320, 4035165321, 4035165327, 4035165336, 4035165337, 4035165354, 4035165365, 4035165400, 4035165410, 4035165424, 4035165440, 4035165441, 4035165449, 4035165454, 4035165458, 4035165460, 4035165465, 4035165475, 4035165495, 4035165499, 4035165508, 4035165532, 4035165549, 4035165558, 4035165559, 4035165572, 4035165573, 4035165577, 4035165585, 4035165587, 4035165598, 4035165625, 4035165626, 4035165627, 4035165634, 4035165639, 4035165645, 4035165653, 4035165663, 4035165665, 4035165666, 4035165676, 4035165678, 4035165682, 4035165687, 4035165695, 4035165701, 4035165702, 4035165708, 4035165724, 4035165725, 4035165742, 4035165744, 4035165745, 4035165755, 4035165758, 4035165768, 4035165774, 4035165781, 4035165782, 4035165783, 4035165796, 4035165797, 4035165804, 4035165812, 4035165813, 4035165832, 4035165836, 4035165839, 4035165847, 4035165848, 4035165849, 4035165864, 4035165870, 4035165904, 4035165905, 4035165909, 4035165913, 4035165915, 4035165918, 4035165926, 4035165939, 4035165958, 4035165969, 4035165972, 4035165973, 4035165976, 4035165993, 4035165997, 4035166004, 4035166023, 4035166042, 4035166056, 4035166061, 4035166080, 4035166094, 4035166101, 4035166104, 4035166109, 4035166111, 4035166124, 4035166150, 4035166206, 4035166208, 4035166213, 4035166227, 4035166229, 4035166231, 4035166239, 4035166249, 4035166251, 4035166275, 4035166311, 4035166322, 4035166348, 4035166350, 4035166351, 4035166363, 4035166376, 4035166378, 4035166383, 4035166395, 4035166396, 4035166398, 4035166412, 4035166446, 4035166450, 4035166458, 4035166460, 4035166490, 4035166507, 4035166512, 4035166524, 4035166549, 4035166552, 4035166554, 4035166565, 4035166595, 4035166597, 4035166599, 4035166601, 4035166606, 4035166608, 4035166621, 4035166626, 4035166640, 4035166644, 4035166656, 4035166668, 4035166669, 4035166670, 4035166706, 4035166708, 4035166722, 4035166724, 4035166750, 4035166752, 4035166759, 4035166765, 4035166786, 4035166806, 4035166807, 4035166814, 4035166826, 4035166828, 4035166841, 4035166842, 4035166844, 4035166849, 4035166854, 4035166868, 4035166885, 4035166889, 4035166897, 4035166917, 4035166948, 4035166963, 4035166964, 4035166972, 4035166974, 4035166976, 4035166992, 4035167026, 4035167040, 4035167044, 4035167051, 4035167060, 4035167068, 4035167069, 4035167072, 4035167073, 4035167076, 4035167077, 4035167079, 4035167085, 4035167100, 4035167110, 4035167113, 4035167116, 4035167122, 4035167144, 4035167148, 4035167168, 4035167188, 4035167191, 4035167192, 4035167213, 4035167219, 4035167226, 4035167228, 4035167231, 4035167234, 4035167241, 4035167257, 4035167267, 4035167276, 4035167284, 4035167287, 4035167288, 4035167328, 4035167348, 4035167354, 4035167381, 4035167399, 4035167403, 4035167405, 4035167426, 4035167435, 4035167453, 4035167479, 4035167485, 4035167503, 4035167506, 4035167520, 4035167521, 4035167522, 4035167523, 4035167527, 4035167534, 4035167535, 4035167537, 4035167551, 4035167555, 4035167558, 4035167575, 4035167583, 4035167607, 4035167608, 4035167622, 4035167628, 4035167630, 4035167632, 4035167638, 4035167639, 4035167640, 4035167648, 4035167650, 4035167666, 4035167667, 4035167682, 4035167687, 4035167701, 4035167704, 4035167711, 4035167720, 4035167723, 4035167725, 4035167727, 4035167730, 4035167732, 4035167759, 4035167772, 4035167778, 4035167782, 4035167783, 4035167795, 4035167801, 4035167805, 4035167821, 4035167825, 4035167848, 4035167857, 4035167860, 4035167878, 4035167884, 4035167885, 4035167891, 4035167922, 4035167926, 4035167936, 4035167943, 4035167947, 4035167970, 4035167982, 4035167985, 4035167986, 4035167989, 4035167995, 4035167997, 4035168009, 4035168018, 4035168019, 4035168032, 4035168035, 4035168037, 4035168042, 4035168046, 4035168049, 4035168054, 4035168072, 4035168103, 4035168117, 4035168119, 4035168124, 4035168128, 4035168136, 4035168138, 4035168139, 4035168142, 4035168144, 4035168146, 4035168158, 4035168168, 4035168194, 4035168196, 4035168213, 4035168227, 4035168230, 4035168241, 4035168256, 4035168281, 4035168289, 4035168297, 4035168298, 4035168299, 4035168308, 4035168316, 4035168328, 4035168348, 4035168353, 4035168360, 4035168365, 4035168375, 4035168377, 4035168405, 4035168411, 4035168414, 4035168433, 4035168449, 4035168460, 4035168488, 4035168493, 4035168494, 4035168497, 4035168509, 4035168515, 4035168521, 4035168523, 4035168526, 4035168537, 4035168560, 4035168575, 4035168609, 4035168612, 4035168613, 4035168627, 4035168635, 4035168643, 4035168664, 4035168670, 4035168671, 4035168674, 4035168685, 4035168703, 4035168706, 4035168708, 4035168714, 4035168723, 4035168737, 4035168777, 4035168779, 4035168780, 4035168782, 4035168785, 4035168798, 4035168805, 4035168813, 4035168832, 4035168845, 4035168851, 4035168863, 4035168877, 4035168887, 4035168888, 4035168891, 4035168894, 4035168901, 4035168903, 4035168913, 4035168914, 4035168915, 4035168926, 4035168939, 4035168948, 4035168965, 4035168967, 4035168997, 4035169002, 4035169003, 4035169014, 4035169016, 4035169061, 4035169066, 4035169082, 4035169089, 4035169097, 4035169105, 4035169120, 4035169121, 4035169125, 4035169126, 4035169127, 4035169136, 4035169144, 4035169146, 4035169157, 4035169159, 4035169160, 4035169163, 4035169173, 4035169191, 4035169196, 4035169210, 4035169224, 4035169227, 4035169233, 4035169257, 4035169258, 4035169263, 4035169287, 4035169290, 4035169294, 4035169295, 4035169303, 4035169307, 4035169308, 4035169332, 4035169333, 4035169340, 4035169342, 4035169343, 4035169344, 4035169358, 4035169359, 4035169362, 4035169390, 4035169394, 4035169398, 4035169402, 4035169414, 4035169419, 4035169427, 4035169434, 4035169452, 4035169469, 4035169485, 4035169496, 4035169514, 4035169519, 4035169530, 4035169536, 4035169547, 4035169559, 4035169576, 4035169654, 4035169689, 4035169690, 4035169694, 4035169695, 4035169698, 4035169703, 4035169709, 4035169723, 4035169733, 4035169743, 4035169757, 4035169758, 4035169771, 4035169787, 4035169814, 4035169831, 4035169832, 4035169878, 4035169898, 4035169909, 4035169923, 4035169932, 4035169938, 4035169951, 4035169956, 4035169963, 4035169966, 4035169972, 4035169989, 4035170000, 4035170004, 4035170024, 4035170032, 4035170033, 4035170044, 4035170046, 4035170047, 4035170052, 4035170061, 4035170065, 4035170072, 4035170075, 4035170082, 4035170088, 4035170101, 4035170159, 4035170175, 4035170227, 4035170231, 4035170258, 4035170276, 4035170295, 4035170311, 4035170314, 4035170327, 4035170330, 4035170355, 4035170357, 4035170366, 4035170383, 4035170384, 4035170385, 4035170435, 4035170438, 4035170443, 4035170451, 4035170454, 4035170471, 4035170486, 4035170489, 4035170490, 4035170498, 4035170499, 4035170514, 4035170523, 4035170538, 4035170552, 4035170553, 4035170555, 4035170582, 4035170640, 4035170662, 4035170698, 4035170710, 4035170713, 4035170715, 4035170728, 4035170754, 4035170761, 4035170773, 4035170774, 4035170804, 4035170808, 4035170813, 4035170815, 4035170816, 4035170842, 4035170852, 4035170855, 4035170862, 4035170883, 4035170885, 4035170904, 4035170916, 4035170921, 4035170927, 4035170929, 4035170935, 4035170942, 4035170948, 4035170959, 4035170993, 4035171019, 4035171045, 4035171047, 4035171048, 4035171051, 4035171075, 4035171092, 4035171099, 4035171106, 4035171132, 4035171137, 4035171138, 4035171139, 4035171142, 4035171158, 4035171165, 4035171175, 4035171198, 4035171205, 4035171224, 4035171247, 4035171250, 4035171253, 4035171257, 4035171262, 4035171266, 4035171273, 4035171276, 4035171286, 4035171288, 4035171289, 4035171292, 4035171293, 4035171295, 4035171304, 4035171305, 4035171310, 4035171313, 4035171338, 4035171339, 4035171368, 4035171394, 4035171412, 4035171431, 4035171433, 4035171434, 4035171442, 4035171451, 4035171470, 4035171476, 4035171510, 4035171511, 4035171512, 4035171513, 4035171523, 4035171524, 4035171528, 4035171551, 4035171555, 4035171557, 4035171561, 4035171562, 4035171564, 4035171576, 4035171577, 4035171578, 4035171607, 4035171610, 4035171622, 4035171638, 4035171641, 4035171662, 4035171665, 4035171667, 4035171669, 4035171677, 4035171685, 4035171693, 4035171696, 4035171700, 4035171701, 4035171711, 4035171719, 4035171729, 4035171732, 4035171740, 4035171773, 4035171774, 4035171778, 4035171790, 4035171791, 4035171794, 4035171811, 4035171826, 4035171846, 4035171851, 4035171873, 4035171882, 4035171889, 4035171891, 4035171892, 4035171896, 4035171920, 4035171921, 4035171940, 4035171961, 4035171962, 4035171967, 4035171980, 4035171993, 4035172001, 4035172004, 4035172012, 4035172015, 4035172028, 4035172035, 4035172037, 4035172040, 4035172044, 4035172052, 4035172053, 4035172059, 4035172060, 4035172065, 4035172087, 4035172089, 4035172091, 4035172094, 4035172103, 4035172121, 4035172142, 4035172149, 4035172153, 4035172154, 4035172169, 4035172183, 4035172185, 4035172201, 4035172209, 4035172226, 4035172233, 4035172240, 4035172241, 4035172255, 4035172280, 4035172281, 4035172309, 4035172312, 4035172316, 4035172317, 4035172318, 4035172320, 4035172321, 4035172322, 4035172338, 4035172346, 4035172350, 4035172354, 4035172365, 4035172375, 4035172391, 4035172399, 4035172418, 4035172454, 4035172460, 4035172463, 4035172469, 4035172488, 4035172505, 4035172506, 4035172544, 4035172549, 4035172580, 4035172583, 4035172599, 4035172602, 4035172615, 4035172654, 4035172655, 4035172689, 4035172708, 4035172732, 4035172740, 4035172743, 4035172745, 4035172757, 4035172781, 4035172784, 4035172788, 4035172795, 4035172823, 4035172827, 4035172831, 4035172835, 4035172836, 4035172860, 4035172863, 4035172864, 4035172878, 4035172894, 4035172901, 4035172902, 4035172916, 4035172917, 4035172923, 4035172928, 4035172940, 4035172945, 4035172953, 4035172955, 4035172982, 4035172990, 4035172992, 4035172997, 4035173016, 4035173023, 4035173030, 4035173041, 4035173044, 4035173065, 4035173069, 4035173089, 4035173112, 4035173117, 4035173122, 4035173126, 4035173136, 4035173141, 4035173150, 4035173152, 4035173155, 4035173167, 4035173180, 4035173184, 4035173188, 4035173198, 4035173210, 4035173218, 4035173225, 4035173226, 4035173237, 4035173242, 4035173254, 4035173255, 4035173256, 4035173268, 4035173270, 4035173288, 4035173289, 4035173308, 4035173309, 4035173310, 4035173318, 4035173339, 4035173341, 4035173356, 4035173358, 4035173367, 4035173380, 4035173426, 4035173457, 4035173463, 4035173474, 4035173479, 4035173527, 4035173545, 4035173560, 4035173574, 4035173579, 4035173595, 4035173618, 4035173619, 4035173628, 4035173631, 4035173638, 4035173646, 4035173647, 4035173674, 4035173680, 4035173681, 4035173693, 4035173694, 4035173709, 4035173721, 4035173728, 4035173757, 4035173767, 4035173788, 4035173796, 4035173806, 4035173820, 4035173826, 4035173827, 4035173830, 4035173852, 4035173862, 4035173882, 4035173885, 4035173921, 4035173935, 4035173952, 4035173960, 4035173973, 4035173974, 4035173978, 4035173985, 4035173986, 4035174003, 4035174004, 4035174032, 4035174047, 4035174048, 4035174049, 4035174053, 4035174066, 4035174083, 4035174099, 4035174100, 4035174102, 4035174103, 4035174104, 4035174108, 4035174128, 4035174138, 4035174141, 4035174145, 4035174149, 4035174162, 4035174165, 4035174173, 4035174192, 4035174207, 4035174210, 4035174212, 4035174214, 4035174237, 4035174242, 4035174251, 4035174260, 4035174264, 4035174279, 4035174281, 4035174282, 4035174293, 4035174297, 4035174301, 4035174304, 4035174318, 4035174319, 4035174321, 4035174334, 4035174338, 4035174371, 4035174375, 4035174379, 4035174387, 4035174391, 4035174410, 4035174417, 4035174421, 4035174441, 4035174457, 4035174460, 4035174461, 4035174467, 4035174468, 4035174475, 4035174488, 4035174491, 4035174495, 4035174496, 4035174500, 4035174507, 4035174541, 4035174555, 4035174566, 4035174598, 4035174610, 4035174620, 4035174621, 4035174630, 4035174644, 4035174660, 4035174663, 4035174676, 4035174679, 4035174681, 4035174696, 4035174709, 4035174710, 4035174712, 4035174726, 4035174727, 4035174732, 4035174750, 4035174759, 4035174765, 4035174777, 4035174794, 4035174810, 4035174815, 4035174846, 4035174854, 4035174869, 4035174880, 4035174885, 4035174893, 4035174894, 4035174898, 4035174909, 4035174917, 4035174923, 4035174941, 4035174947, 4035174958, 4035174969, 4035174986, 4035174988, 4035175003, 4035175014, 4035175017, 4035175020, 4035175024, 4035175043, 4035175052, 4035175070, 4035175096, 4035175104, 4035175106, 4035175117, 4035175122, 4035175123, 4035175128, 4035175130, 4035175138, 4035175152, 4035175153, 4035175160, 4035175187, 4035175198, 4035175199, 4035175202, 4035175206, 4035175216, 4035175226, 4035175238, 4035175246, 4035175252, 4035175257, 4035175270, 4035175273, 4035175284, 4035175285, 4035175293, 4035175312, 4035175319, 4035175326, 4035175348, 4035175365, 4035175369, 4035175371, 4035175385, 4035175398, 4035175417, 4035175419, 4035175428, 4035175431, 4035175433, 4035175436, 4035175472, 4035175479, 4035175489, 4035175530, 4035175531, 4035175533, 4035175558, 4035175561, 4035175562, 4035175594, 4035175602, 4035175635, 4035175645, 4035175659, 4035175665, 4035175686, 4035175693, 4035175713, 4035175714, 4035175731, 4035175741, 4035175750, 4035175815, 4035175824, 4035175826, 4035175830, 4035175837, 4035175853, 4035175861, 4035175872, 4035175873, 4035175875, 4035175890, 4035175904, 4035175905, 4035175907, 4035175916, 4035175919, 4035175926, 4035175933, 4035175936, 4035175945, 4035175948, 4035175958, 4035175961, 4035175962, 4035175981, 4035175983, 4035176001, 4035176016, 4035176020, 4035176034, 4035176035, 4035176114, 4035176121, 4035176125, 4035176134, 4035176148, 4035176150, 4035176156, 4035176161, 4035176163, 4035176164, 4035176171, 4035176172, 4035176182, 4035176184, 4035176202, 4035176204, 4035176212, 4035176223, 4035176229, 4035176261, 4035176273, 4035176279, 4035176281, 4035176285, 4035176297, 4035176298, 4035176300, 4035176303, 4035176313, 4035176318, 4035176319, 4035176332, 4035176336, 4035176338, 4035176341, 4035176342, 4035176354, 4035176356, 4035176361, 4035176363, 4035176378, 4035176383, 4035176393, 4035176413, 4035176415, 4035176419, 4035176420, 4035176424, 4035176437, 4035176455, 4035176460, 4035176481, 4035176482, 4035176489, 4035176495, 4035176504, 4035176507, 4035176512, 4035176515, 4035176541, 4035176544, 4035176555, 4035176581, 4035176593, 4035176652, 4035176664, 4035176666, 4035176667, 4035176682, 4035176697, 4035176699, 4035176705, 4035176728, 4035176740, 4035176743, 4035176747, 4035176750, 4035176751, 4035176769, 4035176776, 4035176778, 4035176792, 4035176795, 4035176803, 4035176827, 4035176828, 4035176829, 4035176844, 4035176855, 4035176859, 4035176867, 4035176876, 4035176877, 4035176886, 4035176892, 4035176899, 4035176908, 4035176916, 4035176932, 4035176934, 4035176940, 4035176945, 4035176948, 4035176982, 4035176983, 4035176990, 4035176992, 4035176995, 4035177017, 4035177034, 4035177040, 4035177051, 4035177059, 4035177063, 4035177088, 4035177089, 4035177111, 4035177146, 4035177148, 4035177152, 4035177173, 4035177191, 4035177196, 4035177219, 4035177225, 4035177235, 4035177248, 4035177259, 4035177261, 4035177292, 4035177294, 4035177298, 4035177300, 4035177303, 4035177309, 4035177315, 4035177319, 4035177325, 4035177335, 4035177337, 4035177338, 4035177360, 4035177366, 4035177374, 4035177388, 4035177391, 4035177396, 4035177407, 4035177416, 4035177422, 4035177436, 4035177481, 4035177497, 4035177513, 4035177545, 4035177551, 4035177555, 4035177583, 4035177592, 4035177623, 4035177629, 4035177637, 4035177644, 4035177669, 4035177676, 4035177687, 4035177696, 4035177706, 4035177707, 4035177716, 4035177729, 4035177730, 4035177737, 4035177751, 4035177755, 4035177790, 4035177803, 4035177810, 4035177829, 4035177831, 4035177873, 4035177889, 4035177903, 4035177914, 4035177917, 4035177931, 4035177954, 4035177960, 4035177966, 4035177968, 4035177971, 4035177979, 4035177985, 4035177987, 4035177988, 4035178027, 4035178030, 4035178038, 4035178059, 4035178063, 4035178083, 4035178084, 4035178097, 4035178098, 4035178106, 4035178116, 4035178146, 4035178158, 4035178166, 4035178169, 4035178170, 4035178177, 4035178204, 4035178206, 4035178222, 4035178223, 4035178230, 4035178240, 4035178246, 4035178252, 4035178264, 4035178273, 4035178278, 4035178295, 4035178298, 4035178304, 4035178316, 4035178334, 4035178339, 4035178342, 4035178347, 4035178364, 4035178377, 4035178391, 4035178396, 4035178402, 4035178408, 4035178413, 4035178438, 4035178445, 4035178449, 4035178451, 4035178478, 4035178505, 4035178517, 4035178520, 4035178526, 4035178528, 4035178535, 4035178538, 4035178543, 4035178545, 4035178550, 4035178565, 4035178575, 4035178584, 4035178586, 4035178590, 4035178612, 4035178620, 4035178626, 4035178657, 4035178673, 4035178674, 4035178675, 4035178702, 4035178703, 4035178728, 4035178732, 4035178733, 4035178750, 4035178758, 4035178790, 4035178802, 4035178821, 4035178835, 4035178840, 4035178842, 4035178855, 4035178856, 4035178863, 4035178875, 4035178890, 4035178891, 4035178895, 4035178909, 4035178919, 4035178927, 4035178942, 4035178966, 4035178970, 4035179000, 4035179002, 4035179020, 4035179022, 4035179023, 4035179030, 4035179048, 4035179049, 4035179072, 4035179073, 4035179081, 4035179099, 4035179117, 4035179121, 4035179122, 4035179125, 4035179140, 4035179145, 4035179152, 4035179170, 4035179191, 4035179192, 4035179210, 4035179218, 4035179227, 4035179228, 4035179229, 4035179234, 4035179238, 4035179244, 4035179251, 4035179252, 4035179257, 4035179259, 4035179262, 4035179268, 4035179272, 4035179281, 4035179282, 4035179292, 4035179298, 4035179299, 4035179307, 4035179319, 4035179325, 4035179337, 4035179345, 4035179377, 4035179379, 4035179392, 4035179400, 4035179406, 4035179414, 4035179415, 4035179416, 4035179436, 4035179448, 4035179455, 4035179462, 4035179478, 4035179480, 4035179481, 4035179516, 4035179517, 4035179528, 4035179541, 4035179553, 4035179559, 4035179580, 4035179585, 4035179616, 4035179617, 4035179625, 4035179633, 4035179635, 4035179642, 4035179643, 4035179669, 4035179670, 4035179671, 4035179683, 4035179701, 4035179702, 4035179721, 4035179763, 4035179772, 4035179782, 4035179787, 4035179800, 4035179829, 4035179846, 4035179861, 4035179874, 4035179875, 4035179879, 4035179883, 4035179901, 4035179902, 4035179906, 4035179913, 4035179917, 4035179920, 4035179921, 4035179929, 4035179931, 4035179932, 4035179935, 4035179942, 4035179946, 4035179962, 4035179970, 4035179971, 4035179996, 4035179999, 4035180004, 4035180009, 4035180018, 4035180025, 4035180026, 4035180049, 4035180064, 4035180067, 4035180071, 4035180078, 4035180082, 4035180089, 4035180099, 4035180114, 4035180126, 4035180137, 4035180138, 4035180140, 4035180146, 4035180160, 4035180162, 4035180168, 4035180175, 4035180187, 4035180219, 4035180240, 4035180248, 4035180275, 4035180282, 4035180288, 4035180290, 4035180292, 4035180296, 4035180298, 4035180300, 4035180312, 4035180329, 4035180333, 4035180339, 4035180340, 4035180347, 4035180352, 4035180362, 4035180363, 4035180370, 4035180382, 4035180389, 4035180394, 4035180419, 4035180420, 4035180436, 4035180443, 4035180445, 4035180448, 4035180451, 4035180479, 4035180481, 4035180487, 4035180501, 4035180511, 4035180515, 4035180516, 4035180529, 4035180559, 4035180575, 4035180576, 4035180597, 4035180600, 4035180606, 4035180611, 4035180639, 4035180647, 4035180670, 4035180672, 4035180693, 4035180706, 4035180710, 4035180713, 4035180731, 4035180737, 4035180744, 4035180802, 4035180810, 4035180827, 4035180833, 4035180838, 4035180841, 4035180845, 4035180851, 4035180857, 4035180869, 4035180871, 4035180874, 4035180880, 4035180884, 4035180891, 4035180892, 4035180909, 4035180916, 4035180918, 4035180926, 4035180935, 4035180976, 4035180978, 4035180981, 4035180983, 4035180984, 4035180990, 4035180998, 4035181007, 4035181028, 4035181034, 4035181038, 4035181047, 4035181063, 4035181071, 4035181078, 4035181102, 4035181103, 4035181108, 4035181123, 4035181125, 4035181127, 4035181133, 4035181144, 4035181163, 4035181165, 4035181167, 4035181179, 4035181202, 4035181207, 4035181220, 4035181228, 4035181231, 4035181240, 4035181257, 4035181273, 4035181280, 4035181285, 4035181294, 4035181298, 4035181299, 4035181304, 4035181307, 4035181308, 4035181314, 4035181318, 4035181335, 4035181342, 4035181343, 4035181380, 4035181389, 4035181396, 4035181421, 4035181432, 4035181435, 4035181442, 4035181443, 4035181444, 4035181456, 4035181466, 4035181485, 4035181496, 4035181501, 4035181502, 4035181504, 4035181507, 4035181508, 4035181512, 4035181516, 4035181531, 4035181548, 4035181550, 4035181561, 4035181563, 4035181588, 4035181598, 4035181600, 4035181610, 4035181617, 4035181622, 4035181629, 4035181631, 4035181642, 4035181643, 4035181653, 4035181659, 4035181672, 4035181677, 4035181678, 4035181686, 4035181687, 4035181697, 4035181716, 4035181724, 4035181743, 4035181779, 4035181780, 4035181785, 4035181794, 4035181795, 4035181808, 4035181812, 4035181843, 4035181850, 4035181855, 4035181861, 4035181872, 4035181887, 4035181888, 4035181889, 4035181895, 4035181921, 4035181924, 4035181925, 4035181933, 4035181943, 4035181947, 4035181953, 4035181960, 4035181979, 4035181999, 4035182003, 4035182015, 4035182019, 4035182024, 4035182052, 4035182053, 4035182093, 4035182114, 4035182121, 4035182131, 4035182135, 4035182142, 4035182163, 4035182170, 4035182174, 4035182182, 4035182185, 4035182194, 4035182201, 4035182205, 4035182215, 4035182228, 4035182229, 4035182235, 4035182242, 4035182248, 4035182255, 4035182259, 4035182278, 4035182284, 4035182285, 4035182293, 4035182295, 4035182301, 4035182330, 4035182333, 4035182343, 4035182355, 4035182362, 4035182379, 4035182381, 4035182396, 4035182407, 4035182424, 4035182470, 4035182476, 4035182483, 4035182486, 4035182488, 4035182493, 4035182502, 4035182508, 4035182509, 4035182512, 4035182517, 4035182525, 4035182534, 4035182544, 4035182564, 4035182565, 4035182580, 4035182602, 4035182607, 4035182611, 4035182614, 4035182617, 4035182619, 4035182626, 4035182629, 4035182640, 4035182642, 4035182648, 4035182651, 4035182652, 4035182656, 4035182665, 4035182682, 4035182687, 4035182701, 4035182710, 4035182712, 4035182714, 4035182728, 4035182732, 4035182741, 4035182779, 4035182791, 4035182792, 4035182804, 4035182807, 4035182809, 4035182832, 4035182835, 4035182836, 4035182845, 4035182846, 4035182858, 4035182861, 4035182865, 4035182898, 4035182899, 4035182901, 4035182935, 4035182963, 4035182978, 4035182983, 4035182984, 4035182996, 4035183019, 4035183045, 4035183046, 4035183073, 4035183076, 4035183077, 4035183078, 4035183087, 4035183108, 4035183128, 4035183132, 4035183136, 4035183144, 4035183158, 4035183163, 4035183165, 4035183180, 4035183191, 4035183213, 4035183219, 4035183234, 4035183263, 4035183266, 4035183267, 4035183268, 4035183271, 4035183272, 4035183284, 4035183325, 4035183328, 4035183330, 4035183338, 4035183340, 4035183346, 4035183365, 4035183368, 4035183374, 4035183379, 4035183392, 4035183404, 4035183406, 4035183409, 4035183424, 4035183426, 4035183428, 4035183431, 4035183435, 4035183439, 4035183442, 4035183446, 4035183447, 4035183459, 4035183470, 4035183488, 4035183536, 4035183553, 4035183556, 4035183577, 4035183590, 4035183601, 4035183605, 4035183621, 4035183623, 4035183635, 4035183636, 4035183644, 4035183646, 4035183648, 4035183652, 4035183653, 4035183678, 4035183700, 4035183701, 4035183702, 4035183732, 4035183740, 4035183745, 4035183763, 4035183766, 4035183772, 4035183778, 4035183781, 4035183784, 4035183786, 4035183799, 4035183823, 4035183831, 4035183835, 4035183836, 4035183855, 4035183858, 4035183864, 4035183865, 4035183866, 4035183868, 4035183870, 4035183893, 4035183894, 4035183905, 4035183914, 4035183917, 4035183922, 4035183934, 4035183939, 4035183961, 4035183963, 4035183967, 4035183978, 4035183984, 4035183991, 4035183994, 4035183995, 4035183997, 4035184008, 4035184024, 4035184027, 4035184029, 4035184070, 4035184074, 4035184097, 4035184114, 4035184116, 4035184117, 4035184120, 4035184122, 4035184128, 4035184142, 4035184156, 4035184159, 4035184170, 4035184185, 4035184190, 4035184195, 4035184230, 4035184235, 4035184237, 4035184239, 4035184243, 4035184251, 4035184264, 4035184266, 4035184278, 4035184291, 4035184304, 4035184312, 4035184315, 4035184332, 4035184352, 4035184366, 4035184367, 4035184385, 4035184394, 4035184407, 4035184416, 4035184421, 4035184430, 4035184433, 4035184444, 4035184453, 4035184463, 4035184466, 4035184471, 4035184474, 4035184481, 4035184482, 4035184496, 4035184497, 4035184498, 4035184519, 4035184528, 4035184537, 4035184547, 4035184581, 4035184587, 4035184607, 4035184610, 4035184625, 4035184629, 4035184639, 4035184661, 4035184665, 4035184679, 4035184680, 4035184689, 4035184691, 4035184692, 4035184697, 4035184715, 4035184717, 4035184719, 4035184736, 4035184746, 4035184759, 4035184764, 4035184767, 4035184771, 4035184810, 4035184819, 4035184826, 4035184828, 4035184837, 4035184847, 4035184869, 4035184879, 4035184880, 4035184883, 4035184898, 4035184911, 4035184912, 4035184930, 4035184949, 4035184952, 4035184967, 4035184974, 4035184978, 4035184997, 4035185025, 4035185039, 4035185044, 4035185051, 4035185062, 4035185064, 4035185068, 4035185073, 4035185074, 4035185096, 4035185100, 4035185106, 4035185136, 4035185148, 4035185153, 4035185155, 4035185182, 4035185185, 4035185191, 4035185205, 4035185206, 4035185224, 4035185254, 4035185272, 4035185274, 4035185282, 4035185288, 4035185291, 4035185307, 4035185308, 4035185325, 4035185327, 4035185341, 4035185344, 4035185352, 4035185391, 4035185393, 4035185395, 4035185435, 4035185452, 4035185454, 4035185455, 4035185457, 4035185458, 4035185468, 4035185474, 4035185493, 4035185510, 4035185512, 4035185520, 4035185546, 4035185552, 4035185562, 4035185565, 4035185572, 4035185579, 4035185582, 4035185589, 4035185608, 4035185642, 4035185659, 4035185662, 4035185663, 4035185686, 4035185713, 4035185715, 4035185724, 4035185725, 4035185730, 4035185731, 4035185751, 4035185773, 4035185784, 4035185794, 4035185800, 4035185808, 4035185814, 4035185820, 4035185821, 4035185830, 4035185865, 4035185866, 4035185868, 4035185879, 4035185882, 4035185888, 4035185889, 4035185892, 4035185895, 4035185900, 4035185905, 4035185906, 4035185916, 4035185925, 4035185935, 4035185943, 4035185945, 4035185954, 4035185985, 4035185988, 4035186005, 4035186008, 4035186009, 4035186014, 4035186042, 4035186043, 4035186047, 4035186049, 4035186074, 4035186081, 4035186092, 4035186099, 4035186100, 4035186110, 4035186126, 4035186149, 4035186189, 4035186190, 4035186195, 4035186198, 4035186200, 4035186203, 4035186204, 4035186214, 4035186216, 4035186243, 4035186251, 4035186254, 4035186276, 4035186286, 4035186287, 4035186291, 4035186313, 4035186317, 4035186340, 4035186341, 4035186350, 4035186354, 4035186369, 4035186384, 4035186385, 4035186404, 4035186410, 4035186416, 4035186424, 4035186436, 4035186448, 4035186471, 4035186474, 4035186480, 4035186512, 4035186514, 4035186515, 4035186530, 4035186532, 4035186535, 4035186536, 4035186550, 4035186564, 4035186565, 4035186568, 4035186579, 4035186602, 4035186603, 4035186626, 4035186627, 4035186628, 4035186631, 4035186646, 4035186650, 4035186655, 4035186661, 4035186669, 4035186703, 4035186705, 4035186732, 4035186737, 4035186742, 4035186743, 4035186746, 4035186749, 4035186751, 4035186773, 4035186795, 4035186798, 4035186805, 4035186818, 4035186829, 4035186830, 4035186839, 4035186845, 4035186856, 4035186862, 4035186865, 4035186878, 4035186885, 4035186888, 4035186899, 4035186910, 4035186911, 4035186918, 4035186926, 4035186933, 4035186941, 4035186945, 4035186949, 4035186958, 4035186960, 4035186966, 4035186972, 4035186974, 4035186976, 4035186977, 4035186988, 4035186989, 4035186991, 4035186996, 4035187009, 4035187017, 4035187052, 4035187063, 4035187073, 4035187075, 4035187093, 4035187105, 4035187106, 4035187109, 4035187123, 4035187124, 4035187133, 4035187140, 4035187142, 4035187146, 4035187150, 4035187160, 4035187165, 4035187175, 4035187182, 4035187209, 4035187212, 4035187219, 4035187224, 4035187225, 4035187242, 4035187245, 4035187250, 4035187254, 4035187255, 4035187256, 4035187259, 4035187268, 4035187281, 4035187282, 4035187302, 4035187303, 4035187315, 4035187319, 4035187334, 4035187347, 4035187357, 4035187365, 4035187367, 4035187384, 4035187388, 4035187393, 4035187398, 4035187405, 4035187421, 4035187429, 4035187433, 4035187451, 4035187464, 4035187489, 4035187490, 4035187501, 4035187502, 4035187507, 4035187515, 4035187520, 4035187537, 4035187545, 4035187566, 4035187594, 4035187601, 4035187604, 4035187615, 4035187620, 4035187644, 4035187645, 4035187674, 4035187679, 4035187687, 4035187693, 4035187694, 4035187712, 4035187727, 4035187728, 4035187732, 4035187738, 4035187741, 4035187762, 4035187785, 4035187798, 4035187801, 4035187803, 4035187831, 4035187839, 4035187841, 4035187847, 4035187850, 4035187862, 4035187865, 4035187877, 4035187884, 4035187888, 4035187901, 4035187904, 4035187912, 4035187928, 4035187932, 4035187940, 4035187941, 4035187971, 4035187988, 4035187997, 4035188045, 4035188053, 4035188056, 4035188100, 4035188103, 4035188124, 4035188133, 4035188150, 4035188160, 4035188170, 4035188178, 4035188188, 4035188208, 4035188213, 4035188216, 4035188218, 4035188228, 4035188229, 4035188240, 4035188244, 4035188254, 4035188268, 4035188274, 4035188287, 4035188329, 4035188337, 4035188344, 4035188351, 4035188369, 4035188377, 4035188387, 4035188388, 4035188389, 4035188395, 4035188397, 4035188402, 4035188411, 4035188422, 4035188447, 4035188448, 4035188471, 4035188492, 4035188494, 4035188497, 4035188521, 4035188546, 4035188558, 4035188563, 4035188564, 4035188566, 4035188567, 4035188570, 4035188597, 4035188599, 4035188627, 4035188630, 4035188631, 4035188643, 4035188649, 4035188652, 4035188655, 4035188666, 4035188673, 4035188675, 4035188686, 4035188690, 4035188701, 4035188710, 4035188712, 4035188713, 4035188724, 4035188739, 4035188745, 4035188786, 4035188798, 4035188812, 4035188813, 4035188815, 4035188819, 4035188823, 4035188839, 4035188841, 4035188843, 4035188845, 4035188846, 4035188849, 4035188866, 4035188875, 4035188876, 4035188878, 4035188880, 4035188895, 4035188901, 4035188904, 4035188906, 4035188932, 4035188939, 4035188956, 4035188968, 4035188969, 4035188984, 4035188987, 4035188989, 4035188990, 4035188994, 4035189001, 4035189014, 4035189015, 4035189063, 4035189069, 4035189070, 4035189084, 4035189090, 4035189099, 4035189134, 4035189145, 4035189146, 4035189163, 4035189187, 4035189188, 4035189195, 4035189204, 4035189210, 4035189214, 4035189228, 4035189231, 4035189248, 4035189250, 4035189252, 4035189255, 4035189280, 4035189302, 4035189308, 4035189311, 4035189316, 4035189324, 4035189328, 4035189345, 4035189358, 4035189361, 4035189366, 4035189367, 4035189369, 4035189374, 4035189377, 4035189381, 4035189409, 4035189419, 4035189422, 4035189440, 4035189448, 4035189461, 4035189488, 4035189489, 4035189498, 4035189509, 4035189518, 4035189519, 4035189521, 4035189523, 4035189545, 4035189547, 4035189551, 4035189556, 4035189562, 4035189572, 4035189577, 4035189579, 4035189581, 4035189594, 4035189626, 4035189628, 4035189629, 4035189640, 4035189659, 4035189660, 4035189663, 4035189669, 4035189677, 4035189679, 4035189686, 4035189692, 4035189693, 4035189704, 4035189708, 4035189715, 4035189719, 4035189725, 4035189728, 4035189741, 4035189744, 4035189757, 4035189791, 4035189792, 4035189813, 4035189849, 4035189869, 4035189870, 4035189871, 4035189877, 4035189897, 4035189913, 4035189917, 4035189919, 4035189928, 4035189929, 4035189941, 4035189942, 4035189945, 4035189951, 4035189964, 4035189970, 4035189972, 4035189977, 4035189991, 4035189997, 4035190020, 4035190029, 4035190032, 4035190039, 4035190059, 4035190083, 4035190098, 4035190103, 4035190109, 4035190114, 4035190165, 4035190166, 4035190167, 4035190174, 4035190175, 4035190179, 4035190181, 4035190182, 4035190183, 4035190187, 4035190189, 4035190204, 4035190205, 4035190210, 4035190212, 4035190248, 4035190256, 4035190273, 4035190276, 4035190290, 4035190324, 4035190338, 4035190343, 4035190349, 4035190355, 4035190365, 4035190369, 4035190387, 4035190388, 4035190401, 4035190408, 4035190409, 4035190411, 4035190418, 4035190419, 4035190424, 4035190428, 4035190435, 4035190464, 4035190483, 4035190484, 4035190485, 4035190501, 4035190503, 4035190509, 4035190530, 4035190540, 4035190549, 4035190553, 4035190579, 4035190586, 4035190600, 4035190620, 4035190646, 4035190656, 4035190672, 4035190674, 4035190677, 4035190683, 4035190692, 4035190702, 4035190717, 4035190730, 4035190743, 4035190752, 4035190755, 4035190756, 4035190768, 4035190772, 4035190773, 4035190779, 4035190782, 4035190811, 4035190815, 4035190818, 4035190828, 4035190867, 4035190871, 4035190875, 4035190904, 4035190963, 4035190982, 4035190983, 4035190996, 4035191009, 4035191019, 4035191025, 4035191034, 4035191040, 4035191041, 4035191047, 4035191048, 4035191054, 4035191055, 4035191056, 4035191057, 4035191060, 4035191096, 4035191104, 4035191110, 4035191111, 4035191144, 4035191146, 4035191152, 4035191153, 4035191154, 4035191162, 4035191166, 4035191176, 4035191177, 4035191189, 4035191202, 4035191212, 4035191220, 4035191233, 4035191252, 4035191253, 4035191275, 4035191278, 4035191301, 4035191307, 4035191323, 4035191334, 4035191347, 4035191367, 4035191378, 4035191402, 4035191408, 4035191415, 4035191441, 4035191445, 4035191465, 4035191468, 4035191508, 4035191523, 4035191534, 4035191553, 4035191564, 4035191573, 4035191590, 4035191600, 4035191606, 4035191616, 4035191617, 4035191635, 4035191636, 4035191672, 4035191682, 4035191701, 4035191709, 4035191715, 4035191743, 4035191776, 4035191780, 4035191786, 4035191795, 4035191803, 4035191821, 4035191831, 4035191834, 4035191841, 4035191849, 4035191854, 4035191860, 4035191870, 4035191871, 4035191874, 4035191876, 4035191893, 4035191912, 4035191920, 4035191925, 4035191926, 4035191939, 4035191953, 4035191963, 4035191964, 4035191978, 4035191984, 4035191986, 4035191993, 4035191998, 4035192009, 4035192023, 4035192051, 4035192062, 4035192080, 4035192101, 4035192104, 4035192122, 4035192154, 4035192167, 4035192171, 4035192172, 4035192176, 4035192177, 4035192178, 4035192192, 4035192206, 4035192207, 4035192210, 4035192217, 4035192243, 4035192274, 4035192304, 4035192309, 4035192331, 4035192334, 4035192338, 4035192341, 4035192348, 4035192349, 4035192372, 4035192381, 4035192390, 4035192405, 4035192410, 4035192414, 4035192418, 4035192446, 4035192462, 4035192503, 4035192521, 4035192541, 4035192545, 4035192547, 4035192564, 4035192575, 4035192581, 4035192591, 4035192593, 4035192594, 4035192605, 4035192614, 4035192615, 4035192636, 4035192638, 4035192652, 4035192653, 4035192665, 4035192666, 4035192676, 4035192678, 4035192688, 4035192693, 4035192720, 4035192728, 4035192741, 4035192743, 4035192747, 4035192757, 4035192773, 4035192784, 4035192786, 4035192797, 4035192807, 4035192808, 4035192814, 4035192821, 4035192825, 4035192830, 4035192859, 4035192868, 4035192877, 4035192885, 4035192905, 4035192912, 4035192914, 4035192918, 4035192927, 4035192945, 4035192948, 4035192969, 4035192970, 4035192987, 4035192988, 4035192991, 4035192995, 4035192999, 4035193002, 4035193008, 4035193011, 4035193019, 4035193025, 4035193036, 4035193072, 4035193089, 4035193105, 4035193117, 4035193118, 4035193138, 4035193177, 4035193180, 4035193181, 4035193208, 4035193218, 4035193219, 4035193256, 4035193265, 4035193274, 4035193281, 4035193293, 4035193302, 4035193309, 4035193322, 4035193325, 4035193327, 4035193342, 4035193345, 4035193354, 4035193359, 4035193381, 4035193384, 4035193399, 4035193401, 4035193423, 4035193426, 4035193428, 4035193446, 4035193450, 4035193452, 4035193459, 4035193460, 4035193465, 4035193467, 4035193485, 4035193508, 4035193520, 4035193529, 4035193542, 4035193565, 4035193568, 4035193576, 4035193579, 4035193581, 4035193588, 4035193594, 4035193598, 4035193607, 4035193608, 4035193616, 4035193640, 4035193660, 4035193669, 4035193680, 4035193685, 4035193687, 4035193699, 4035193708, 4035193718, 4035193728, 4035193734, 4035193775, 4035193791, 4035193794, 4035193795, 4035193810, 4035193811, 4035193818, 4035193828, 4035193835, 4035193850, 4035193854, 4035193855, 4035193871, 4035193877, 4035193885, 4035193888, 4035193893, 4035193899, 4035193927, 4035193941, 4035193960, 4035193964, 4035193965, 4035193969, 4035194025, 4035194026, 4035194041, 4035194059, 4035194075, 4035194076, 4035194111, 4035194115, 4035194133, 4035194136, 4035194142, 4035194143, 4035194144, 4035194149, 4035194161, 4035194184, 4035194186, 4035194193, 4035194207, 4035194210, 4035194236, 4035194238, 4035194242, 4035194246, 4035194250, 4035194252, 4035194255, 4035194264, 4035194267, 4035194280, 4035194298, 4035194308, 4035194309, 4035194319, 4035194322, 4035194334, 4035194344, 4035194354, 4035194362, 4035194377, 4035194383, 4035194387, 4035194399, 4035194415, 4035194432, 4035194435, 4035194449, 4035194461, 4035194465, 4035194470, 4035194483, 4035194484, 4035194501, 4035194524, 4035194542, 4035194543, 4035194547, 4035194550, 4035194551, 4035194580, 4035194581, 4035194589, 4035194600, 4035194608, 4035194611, 4035194613, 4035194619, 4035194624, 4035194634, 4035194636, 4035194641, 4035194642, 4035194650, 4035194655, 4035194661, 4035194664, 4035194665, 4035194666, 4035194673, 4035194678, 4035194689, 4035194690, 4035194693, 4035194714, 4035194721, 4035194736, 4035194745, 4035194748, 4035194762, 4035194770, 4035194778, 4035194782, 4035194787, 4035194801, 4035194804, 4035194807, 4035194813, 4035194815, 4035194817, 4035194832, 4035194846, 4035194887, 4035194900, 4035194901, 4035194906, 4035194907, 4035194924, 4035194954, 4035194958, 4035194967, 4035194970, 4035194999, 4035195007, 4035195025, 4035195051, 4035195079, 4035195101, 4035195104, 4035195121, 4035195147, 4035195181, 4035195196, 4035195204, 4035195227, 4035195231, 4035195236, 4035195247, 4035195258, 4035195264, 4035195271, 4035195279, 4035195295, 4035195296, 4035195315, 4035195329, 4035195330, 4035195334, 4035195344, 4035195352, 4035195373, 4035195384, 4035195409, 4035195420, 4035195431, 4035195435, 4035195436, 4035195462, 4035195464, 4035195465, 4035195484, 4035195507, 4035195514, 4035195518, 4035195527, 4035195528, 4035195529, 4035195538, 4035195548, 4035195549, 4035195566, 4035195570, 4035195592, 4035195593, 4035195609, 4035195615, 4035195617, 4035195619, 4035195647, 4035195649, 4035195654, 4035195658, 4035195678, 4035195690, 4035195702, 4035195709, 4035195719, 4035195731, 4035195733, 4035195735, 4035195739, 4035195750, 4035195755, 4035195759, 4035195769, 4035195786, 4035195793, 4035195795, 4035195796, 4035195801, 4035195803, 4035195808, 4035195811, 4035195813, 4035195827, 4035195837, 4035195841, 4035195843, 4035195844, 4035195858, 4035195862, 4035195866, 4035195870, 4035195879, 4035195883, 4035195886, 4035195893, 4035195899, 4035195900, 4035195908, 4035195920, 4035195924, 4035195929, 4035195933, 4035195938, 4035195953, 4035195971, 4035195976, 4035196003, 4035196007, 4035196010, 4035196011, 4035196029, 4035196034, 4035196040, 4035196069, 4035196078, 4035196089, 4035196101, 4035196104, 4035196135, 4035196140, 4035196149, 4035196161, 4035196166, 4035196187, 4035196191, 4035196196, 4035196203, 4035196207, 4035196215, 4035196239, 4035196242, 4035196244, 4035196248, 4035196251, 4035196263, 4035196267, 4035196289, 4035196293, 4035196298, 4035196300, 4035196320, 4035196327, 4035196337, 4035196339, 4035196356, 4035196390, 4035196391, 4035196397, 4035196400, 4035196401, 4035196414, 4035196441, 4035196448, 4035196458, 4035196462, 4035196472, 4035196473, 4035196482, 4035196503, 4035196511, 4035196532, 4035196541, 4035196566, 4035196570, 4035196575, 4035196581, 4035196589, 4035196602, 4035196614, 4035196650, 4035196695, 4035196701, 4035196709, 4035196733, 4035196737, 4035196739, 4035196740, 4035196750, 4035196756, 4035196763, 4035196766, 4035196770, 4035196776, 4035196782, 4035196803, 4035196804, 4035196806, 4035196841, 4035196845, 4035196846, 4035196867, 4035196877, 4035196881, 4035196893, 4035196901, 4035196908, 4035196917, 4035196919, 4035196927, 4035196950, 4035196953, 4035196957, 4035196972, 4035196993, 4035196995, 4035196999, 4035197000, 4035197019, 4035197025, 4035197026, 4035197038, 4035197043, 4035197048, 4035197060, 4035197076, 4035197089, 4035197098, 4035197148, 4035197150, 4035197157, 4035197170, 4035197175, 4035197180, 4035197183, 4035197187, 4035197196, 4035197202, 4035197208, 4035197244, 4035197276, 4035197305, 4035197319, 4035197321, 4035197322, 4035197337, 4035197339, 4035197343, 4035197348, 4035197362, 4035197364, 4035197376, 4035197411, 4035197416, 4035197419, 4035197421, 4035197422, 4035197432, 4035197436, 4035197438, 4035197451, 4035197492, 4035197503, 4035197506, 4035197517, 4035197535, 4035197551, 4035197560, 4035197589, 4035197608, 4035197618, 4035197633, 4035197634, 4035197635, 4035197646, 4035197659, 4035197660, 4035197667, 4035197680, 4035197687, 4035197707, 4035197714, 4035197740, 4035197759, 4035197768, 4035197778, 4035197795, 4035197803, 4035197819, 4035197834, 4035197841, 4035197849, 4035197868, 4035197869, 4035197876, 4035197886, 4035197902, 4035197916, 4035197935, 4035197950, 4035197954, 4035197961, 4035197963, 4035197982, 4035197997, 4035198017, 4035198027, 4035198028, 4035198032, 4035198040, 4035198049, 4035198054, 4035198056, 4035198065, 4035198077, 4035198082, 4035198099, 4035198102, 4035198103, 4035198104, 4035198107, 4035198135, 4035198140, 4035198158, 4035198163, 4035198168, 4035198169, 4035198177, 4035198188, 4035198195, 4035198197, 4035198199, 4035198202, 4035198203, 4035198205, 4035198213, 4035198217, 4035198226, 4035198229, 4035198238, 4035198241, 4035198256, 4035198277, 4035198280, 4035198301, 4035198326, 4035198343, 4035198350, 4035198357, 4035198358, 4035198369, 4035198380, 4035198427, 4035198432, 4035198434, 4035198438, 4035198446, 4035198451, 4035198460, 4035198474, 4035198480, 4035198488, 4035198514, 4035198518, 4035198526, 4035198535, 4035198536, 4035198562, 4035198574, 4035198601, 4035198608, 4035198611, 4035198635, 4035198639, 4035198670, 4035198680, 4035198723, 4035198746, 4035198756, 4035198771, 4035198772, 4035198804, 4035198809, 4035198817, 4035198818, 4035198830, 4035198833, 4035198848, 4035198854, 4035198861, 4035198875, 4035198876, 4035198887, 4035198892, 4035198893, 4035198894, 4035198895, 4035198896, 4035198905, 4035198919, 4035198941, 4035198971, 4035198982, 4035198983, 4035198995, 4035199007, 4035199009, 4035199012, 4035199014, 4035199048, 4035199069, 4035199074, 4035199084, 4035199086, 4035199096, 4035199120, 4035199123, 4035199130, 4035199137, 4035199150, 4035199179, 4035199198, 4035199203, 4035199204, 4035199215, 4035199224, 4035199231, 4035199239, 4035199243, 4035199260, 4035199268, 4035199280, 4035199285, 4035199287, 4035199293, 4035199298, 4035199309, 4035199317, 4035199321, 4035199326, 4035199342, 4035199349, 4035199356, 4035199360, 4035199384, 4035199399, 4035199415, 4035199436, 4035199445, 4035199453, 4035199474, 4035199483, 4035199485, 4035199513, 4035199518, 4035199525, 4035199556, 4035199562, 4035199565, 4035199568, 4035199570, 4035199587, 4035199603, 4035199613, 4035199630, 4035199655, 4035199660, 4035199671, 4035199680, 4035199681, 4035199694, 4035199697, 4035199718, 4035199726, 4035199736, 4035199738, 4035199744, 4035199746, 4035199760, 4035199761, 4035199766, 4035199771, 4035199782, 4035199792, 4035199812, 4035199844, 4035199846, 4035199854, 4035199870, 4035199872, 4035199873, 4035199900, 4035199911, 4035199924, 4035199925, 4035199942, 4035199945, 4035199946, 4035199950, 4035199951, 4035199960, 4035199961, 4035199973, 4035199979, 4035199981, 4035199984, 4035199986, 4035200000, 4035200003, 4035200005, 4035200020, 4035200026, 4035200029, 4035200035, 4035200043, 4035200046, 4035200050, 4035200063, 4035200068, 4035200093, 4035200110, 4035200124, 4035200129, 4035200132, 4035200140, 4035200146, 4035200151, 4035200175, 4035200191, 4035200210, 4035200215, 4035200245, 4035200249, 4035200259, 4035200290, 4035200303, 4035200324, 4035200329, 4035200340, 4035200348, 4035200349, 4035200361, 4035200380, 4035200383, 4035200385, 4035200386, 4035200424, 4035200436, 4035200438, 4035200444, 4035200461, 4035200473, 4035200507, 4035200521, 4035200522, 4035200529, 4035200534, 4035200536, 4035200538, 4035200540, 4035200564, 4035200578, 4035200588, 4035200601, 4035200621, 4035200656, 4035200658, 4035200661, 4035200664, 4035200676, 4035200677, 4035200679, 4035200688, 4035200710, 4035200715, 4035200740, 4035200741, 4035200743, 4035200744, 4035200746, 4035200747, 4035200751, 4035200760, 4035200777, 4035200778, 4035200779, 4035200788, 4035200797, 4035200801, 4035200807, 4035200809, 4035200825, 4035200828, 4035200835, 4035200844, 4035200863, 4035200878, 4035200893, 4035200895, 4035200907, 4035200930, 4035200934, 4035200943, 4035200960, 4035200966, 4035200968, 4035200977, 4035200992, 4035200997, 4035201012, 4035201017, 4035201043, 4035201053, 4035201064, 4035201065, 4035201068, 4035201076, 4035201077, 4035201101, 4035201156, 4035201159, 4035201167, 4035201168, 4035201171, 4035201181, 4035201190, 4035201214, 4035201228, 4035201247, 4035201251, 4035201269, 4035201275, 4035201283, 4035201295, 4035201296, 4035201303, 4035201320, 4035201325, 4035201326, 4035201347, 4035201385, 4035201400, 4035201415, 4035201419, 4035201425, 4035201426, 4035201429, 4035201430, 4035201431, 4035201441, 4035201462, 4035201466, 4035201472, 4035201483, 4035201499, 4035201511, 4035201528, 4035201538, 4035201548, 4035201550, 4035201551, 4035201556, 4035201559, 4035201561, 4035201562, 4035201564, 4035201571, 4035201580, 4035201585, 4035201598, 4035201599, 4035201600, 4035201607, 4035201626, 4035201630, 4035201636, 4035201642, 4035201651, 4035201662, 4035201673, 4035201675, 4035201678, 4035201682, 4035201693, 4035201703, 4035201722, 4035201723, 4035201727, 4035201734, 4035201753, 4035201754, 4035201758, 4035201759, 4035201767, 4035201799, 4035201807, 4035201809, 4035201810, 4035201819, 4035201823, 4035201830, 4035201840, 4035201846, 4035201864, 4035201867, 4035201870, 4035201876, 4035201901, 4035201920, 4035201931, 4035201935, 4035201936, 4035201940, 4035201961, 4035201964, 4035201970, 4035201990, 4035202002, 4035202012, 4035202019, 4035202026, 4035202044, 4035202066, 4035202073, 4035202081, 4035202087, 4035202111, 4035202114, 4035202116, 4035202117, 4035202118, 4035202132, 4035202138, 4035202153, 4035202163, 4035202164, 4035202172, 4035202173, 4035202182, 4035202183, 4035202202, 4035202206, 4035202210, 4035202217, 4035202260, 4035202262, 4035202263, 4035202265, 4035202272, 4035202298, 4035202301, 4035202311, 4035202334, 4035202350, 4035202354, 4035202376, 4035202383, 4035202402, 4035202404, 4035202410, 4035202415, 4035202429, 4035202445, 4035202473, 4035202536, 4035202556, 4035202585, 4035202594, 4035202597, 4035202598, 4035202602, 4035202607, 4035202616, 4035202619, 4035202626, 4035202627, 4035202633, 4035202642, 4035202655, 4035202658, 4035202663, 4035202672, 4035202689, 4035202703, 4035202704, 4035202707, 4035202718, 4035202722, 4035202729, 4035202734, 4035202735, 4035202737, 4035202744, 4035202749, 4035202775, 4035202790, 4035202797, 4035202798, 4035202811, 4035202812, 4035202840, 4035202842, 4035202861, 4035202867, 4035202872, 4035202886, 4035202888, 4035202924, 4035202936, 4035202948, 4035202949, 4035202954, 4035202956, 4035202985, 4035202991, 4035203001, 4035203002, 4035203023, 4035203025, 4035203037, 4035203039, 4035203051, 4035203052, 4035203080, 4035203085, 4035203110, 4035203133, 4035203151, 4035203152, 4035203157, 4035203163, 4035203172, 4035203177, 4035203211, 4035203219, 4035203221, 4035203240, 4035203254, 4035203263, 4035203268, 4035203271, 4035203275, 4035203276, 4035203298, 4035203312, 4035203323, 4035203334, 4035203339, 4035203341, 4035203362, 4035203403, 4035203407, 4035203416, 4035203443, 4035203451, 4035203480, 4035203490, 4035203496, 4035203497, 4035203514, 4035203519, 4035203522, 4035203523, 4035203575, 4035203578, 4035203579, 4035203585, 4035203586, 4035203589, 4035203594, 4035203617, 4035203620, 4035203624, 4035203640, 4035203646, 4035203648, 4035203654, 4035203666, 4035203670, 4035203676, 4035203681, 4035203686, 4035203693, 4035203699, 4035203703, 4035203704, 4035203716, 4035203718, 4035203748, 4035203768, 4035203773, 4035203776, 4035203803, 4035203806, 4035203811, 4035203822, 4035203832, 4035203833, 4035203844, 4035203845, 4035203849, 4035203875, 4035203876, 4035203881, 4035203882, 4035203907, 4035203926, 4035203933, 4035203936, 4035203937, 4035203939, 4035203952, 4035203969, 4035203973, 4035203977, 4035203983, 4035203984, 4035203992, 4035204039, 4035204049, 4035204054, 4035204071, 4035204080, 4035204081, 4035204085, 4035204136, 4035204141, 4035204146, 4035204154, 4035204168, 4035204177, 4035204178, 4035204183, 4035204184, 4035204186, 4035204219, 4035204221, 4035204235, 4035204243, 4035204265, 4035204274, 4035204276, 4035204285, 4035204290, 4035204293, 4035204312, 4035204333, 4035204345, 4035204348, 4035204353, 4035204361, 4035204376, 4035204387, 4035204388, 4035204389, 4035204405, 4035204412, 4035204416, 4035204443, 4035204449, 4035204455, 4035204461, 4035204465, 4035204488, 4035204506, 4035204512, 4035204516, 4035204521, 4035204543, 4035204549, 4035204556, 4035204562, 4035204568, 4035204573, 4035204641, 4035204650, 4035204666, 4035204686, 4035204687, 4035204704, 4035204705, 4035204708, 4035204729, 4035204731, 4035204751, 4035204781, 4035204789, 4035204799, 4035204800, 4035204802, 4035204816, 4035204823, 4035204828, 4035204841, 4035204844, 4035204866, 4035204874, 4035204876, 4035204889, 4035204893, 4035204903, 4035204908, 4035204913, 4035204917, 4035204924, 4035204926, 4035204942, 4035204948, 4035204964, 4035204969, 4035204981, 4035204982, 4035204987, 4035204992, 4035204999, 4035205002, 4035205014, 4035205017, 4035205022, 4035205031, 4035205066, 4035205077, 4035205085, 4035205099, 4035205100, 4035205121, 4035205130, 4035205140, 4035205141, 4035205162, 4035205163, 4035205166, 4035205167, 4035205178, 4035205190]

In [150]:
json=get_match_json("NA1_"+str(matchidlist[5000]))
#json=={'status': {'message': 'Data not found - match file not found',
  #'status_code': 404}}
json

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4035156356',
  'participants': ['hkRVnTbozetcLOpLsCQU5S6ccjNotIpiuK2rtcnzUQRm1Sx-x007h0SxqrPJO4v3zJFJ390dNh-Y3w',
   'aoMVkSwe7D-wVwpISa7abDPr8wPIfY_sSSVCX9fnNUJgohwIjY7I1EoZqTgDU8T3NMkPm3CTtSaHGw',
   'K0IfbmVoQyCe3_DLxlnBqZInM08t05n393aAc1Ho496_n_Rtq14lC2fD29r8ifHv3G8y3NAMcQLC8Q',
   'mi2cAhitYGb8HoqXo6r3FyUxmXBXTig6iN4ziyzwSFpJB1h1PrIrIBeRY6jUhJ5BttdHIo4ofmP9Hw',
   'WyC5_wX88yvsqIdLR0INhHz1IlTrMwd41NGbyEp22C8OQRtojUcldp_ckqHWDf37ZVVEW0RMvH0pqQ',
   'xgAIkrgaxBdnkZR0F8jlIez_OSiCn-9tY4E_CutvM-JQLrI8zCMynPlOHg7Qo04WpZS_uvcRzSgAxA',
   'vkGiJxXA3NYUYia670CWqIqRGeWU94P9DOT4x8-nzOXlDFfKmJ2E-JwDvIQJBqUaBbFYkEA0Fb19Ww',
   '78LIXP67Y6e0Ztv-ZFZRyHLl6thPuUKyBMXY8hDc3yUbNlSVO8zH62UABewjKMKKoeN0htJSDf7OWA',
   'h92gRxUwv1itZHn01htWTNJNFEXfVUg4joJLkQD6hIMsijNNjbIbq6CDC35yVrcaJuBBh-ugXlLfZQ',
   'G15ftZu_Qww_-ZGuBlnMG1TdFwvtuAT1v5qvhFQ7xWfRfiSjvfPYS2ZbBfBRDeDMB43dOpaMERkPaw']},
 'info': {'frameInterval': 60000,
  'frames': [{'events': [{'realTi

In [12]:
wholedf=get_1match_datav2(matchidlist[:5])

finish: 1
finish: 2
finish: 3
finish: 4
finish: 5


In [14]:
wholedf

,golddiff,dragondiff,barondiff,heralddiff,towerdiff,inhibitordiff,killdiff,winner
0,"[0, 0, -1720, -1523, -1169, -618, -723, -522, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 2, 1, 0, -1, -2, -2, -3, -3, -...",0.0
1,"[0, 0, 0, 144, 627, -115, 5, 401, 596, 1107, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -2, -...",1.0
2,"[0, 0, 0, 208, 634, 1031, 1445, 1671, 2157, 27...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, -1, -4, -3, -3, -5, -6, -4,...",1.0
3,"[0, 0, 375, 514, 990, 2031, 2618, 2680, 3406, ...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 1, 2, 3, 3, 4, 3, 3, 3, 4, 4, 4, ...",1.0
4,"[0, 0, -240, -912, 191, 48, 648, 1041, 1395, 1...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 3, 3, 3, 4, 4, 3, 4, 4, 6, 7, 6, 7, ...",0.0


In [23]:
wholedf.to_csv('visdata_5.csv', index=False)